---
title: "DRAFT: PyTorch and Python Free-Threading"
categories:
  - PyTorch
  - Python
  - Free-Threading
  - No-GIL
  - LLM
  - GPT2
author: "Trent Nelson"
draft: true
image: "images/pytorch-and-python-free-threading.png"
description: |
  This blog post explores multi-threaded PyTorch parallel inference via an
  `asyncio`-based HTTP server and the new *No-GIL* free-threaded Python,
  using a (predominantly-AI-written) simple React Bootstrap web interface.
  We capture what is currently possible, what type of issues are typically
  encountered, and what the roadmap may look like for the future.
jupyter:
  kernelspec:
    display_name: "Python 3.13t (Free-Threaded)"
    language: python
    name: py313t
---


This blog post is sponsored in part by [Meta](https://meta.com) in
collaboration with [Quansight](https://quansight.com) and
[OpenTeams](https://openteams.com).

# Introduction

Python 3.13, released in October 2024, is the first version of Python to
introduce support for a "no-GIL" *free-threaded* mode, per
[PEP-703 Making the Global Interpreter Lock Optional in CPython](
https://peps.python.org/pep-0703/), unlocking the ability for multiple Python
threads to run simultaneously.

This allows, for the first time since the language's inception in December
1989, a single Python process to saturate all CPU cores in parallel with
pure Python code (i.e. not farming out to extension modules written in C, C++,
or, more recently, Rust).

A handful of the [motivations](https://peps.python.org/pep-0703/#motivation)
captured in that PEP opine on how the GIL impedes Python AI workflows,
particularly as it relates to GPU programming.

This blog post explores what can be done with [PyTorch](https://pytorch.org)
now with the new free-threaded version of Python, specifically focusing on
run-time inference on transformer-based generative models.  Using a simple
React Bootstrap web interface for the front-end, a pure-Python
`asyncio`-based multi-threaded HTTP server is used to facilitate
multi-threaded, simultaneous parallel model inference.

# Getting Started

All of this work was done on Linux (Ubuntu 22.04) with Python 3.13t, PyTorch
2.6, and CUDA 12.6.  I would have liked to get Windows support working too,
but unfortunately, the multi-threaded `asyncio`-based HTTP server I wrote
for Linux doesn't appear to leverage multiple threads on Windows.  (From a
cursory inspection, it appears to be an issue either with event loops not
getting created properly on non-main-thread threads, or a more insidious
issue with how the `IocpProactor()` code uses I/O completion ports.)

## Environments

In general, I endeavored to minimize the number of external library
dependencies as best I could.  The primary reason for this is that a lot of
the modern Python AI stack isn't yet compatible with Python free-threaded
builds, unfortunately.  This is improving at a rapid pace, though.

Anything written in Rust is particularly problematic, as
[PyO3](https://github.com/PyO3/pyo3) (Rust bindings for Python) only recently
supported free-threaded Python 3.13+ as of 0.23.3, which was released early
December, 2025, and a lot of projects haven't yet updated to it (like TikToken
and Pydantic).  Other common projects like `lxml` aren't compatible yet either,
which means you can't simply do `pip install llama-stack` and start playing
around.  Likewise for `transformers` and other parts of the HuggingFace stack.

I will reference two `conda` environments in this post: a Python 3.13
free-threaded one named `py313t`, and a normal, not-free-threaded Python
3.13 one named `py313`.

The primary motivation behind the second `py313` environment is it allows
us to install Jupyter Lab, which, at the time of writing, still isn't
compatible with a Python free-threaded installation.  However, we can still
register a free-threaded Python kernel with Jupyter, which is all we really
care about when running the code in this post in a free-threaded environment.

Details on creating the `conda` environments follow.

### Free-Threaded 3.13 Env (py313t)

I use `conda` to create the Python 3.13 free-threaded environment plus
initial dependencies, activate it, then install the remaining dependencies
via pip, as follows:

```bash
conda create -n py313t python=3.13 python-freethreading \
    nodejs pip tqdm flake8 rust \
        -c conda-forge
conda activate py313t
pip install numpy setuptools_rust regex requests datrie
```

`nodejs` is required for the UI component we'll introduce later.  `regex`,
`rust`, and `setuptools_rust` are needed for `tiktoken`, described next.
Finally, `numpy` is for `torch`, which we install later, too.

#### TikToken

[TikToken](https://github.com/openai/tiktoken) is a fast BPE tokenizer from
OpenAI that is used extensively in the emerging Python LLM landscape.  At the
time of writing, the latest TikToken release was [0.8.0](
https://github.com/openai/tiktoken/releases/tag/0.8.0), which was built
against PyO3 0.22.2, which isn't compatible with free-threaded Python.

Thankfully, it was trivial to get a local installation of `tiktoken` working
by cloning the Github repo, bumping the PyO3 version in `Cargo.toml`, then
rebuilding and installing.

::: {.callout-note}

This is a perfect example of the type of fiddling around I wanted to avoid
by not depending on any external packages other than the bare necessities,
such as PyTorch.  I made an exception for `tiktoken` because a) it's arguably
an equally-important part of the LLM stack as `torch`, and b) it thankfully
wasn't *too* difficult getting a compatible version of `tiktoken` installed
locally.

:::

Clone the tiktoken git repo and cd into it as follows:

```bash
git clone https://github.com/openai/tiktoken
cd tiktoken
```

Edit the `Cargo.toml` file and change the `pyo3` dependency version to at
least 0.23.3^[I used 0.23.3, as that was the latest version available at the
time, however, 0.23.4 has since been released, so you could try that too]:

```diff
diff --git a/Cargo.toml b/Cargo.toml
index 2eed0c1..6be5f63 100644
--- a/Cargo.toml
+++ b/Cargo.toml
@@ -9,7 +9,7 @@ name = "_tiktoken"
 crate-type = ["cdylib"]
 
 [dependencies]
-pyo3 = { version = "0.22.2", default-features = false, features = ["extension-module", "macros"] }
+pyo3 = { version = "0.23.3", default-features = false, features = ["extension-module", "macros"] }
 
 # tiktoken dependencies
 fancy-regex = "0.13.0"
```

With this patch applied, and the `py313t` conda environment active (with
`rust` and `setuptools_rust` installed):

```bash
python setup.py build
python setup.py install
```

::: {.callout-note collapse="true" appearance="simple" icon="false" title="Show Build & Install Output"}


```bash
(py313t) {pytorch} [12.6] [trent@dgx/ttypts/3(~s/tiktoken)%] python setup.py build
running build
running build_py
copying tiktoken/_educational.py -> build/lib.linux-x86_64-cpython-313t/tiktoken
copying tiktoken/__init__.py -> build/lib.linux-x86_64-cpython-313t/tiktoken
copying tiktoken/registry.py -> build/lib.linux-x86_64-cpython-313t/tiktoken
copying tiktoken/model.py -> build/lib.linux-x86_64-cpython-313t/tiktoken
copying tiktoken/load.py -> build/lib.linux-x86_64-cpython-313t/tiktoken
copying tiktoken/core.py -> build/lib.linux-x86_64-cpython-313t/tiktoken
copying tiktoken_ext/openai_public.py -> build/lib.linux-x86_64-cpython-313t/tiktoken_ext
running egg_info
writing tiktoken.egg-info/PKG-INFO
writing dependency_links to tiktoken.egg-info/dependency_links.txt
writing requirements to tiktoken.egg-info/requires.txt
writing top-level names to tiktoken.egg-info/top_level.txt
reading manifest file 'tiktoken.egg-info/SOURCES.txt'
reading manifest template 'MANIFEST.in'
warning: no files found matching 'Makefile'
adding license file 'LICENSE'
writing manifest file 'tiktoken.egg-info/SOURCES.txt'
copying tiktoken/py.typed -> build/lib.linux-x86_64-cpython-313t/tiktoken
running build_ext
running build_rust
cargo rustc --lib --message-format=json-render-diagnostics --manifest-path Cargo.toml --release -v --features pyo3/extension-module --crate-type cdylib --
   Compiling target-lexicon v0.12.16
   Compiling once_cell v1.20.2
   Compiling proc-macro2 v1.0.92
   Compiling unicode-ident v1.0.14
   Compiling memchr v2.7.4
   Compiling regex-syntax v0.8.5
   Compiling libc v0.2.169
   Compiling autocfg v1.4.0
   Compiling heck v0.5.0
   Compiling bit-vec v0.6.3
   Compiling unindent v0.2.3
   Compiling indoc v2.0.5
   Compiling cfg-if v1.0.0
   Compiling rustc-hash v1.1.0
     Running `rustc --crate-name build_script_build --edition=2018 /home/trent/.cargo/registry/src/index.crates.io-6f17d22bba15001f/target-lexicon-0.12.16/build.rs --error-format=json --json=diagnostic-rendered-ansi,artifacts,future-incompat --diagnostic-width=254 --crate-type bin --emit=dep-info,link -C embed-bitcode=no -C debug-assertions=off --cfg 'feature="default"' --check-cfg 'cfg(docsrs)' --check-cfg 'cfg(feature, values("arch_zkasm", "default", "serde", "serde_support", "std"))' -C metadata=826df8be4fa9ef21 -C extra-filename=-826df8be4fa9ef21 --out-dir /home/trent/src/tiktoken/target/release/build/target-lexicon-826df8be4fa9ef21 -C linker=/home/trent/mambaforge/envs/py313t/bin/x86_64-conda-linux-gnu-cc -C strip=debuginfo -L dependency=/home/trent/src/tiktoken/target/release/deps --cap-lints allow`
     Running `rustc --crate-name once_cell --edition=2021 /home/trent/.cargo/registry/src/index.crates.io-6f17d22bba15001f/once_cell-1.20.2/src/lib.rs --error-format=json --json=diagnostic-rendered-ansi,artifacts,future-incompat --diagnostic-width=254 --crate-type lib --emit=dep-info,metadata,link -C embed-bitcode=no -C debug-assertions=off --cfg 'feature="alloc"' --cfg 'feature="default"' --cfg 'feature="race"' --cfg 'feature="std"' --check-cfg 'cfg(docsrs)' --check-cfg 'cfg(feature, values("alloc", "atomic-polyfill", "critical-section", "default", "parking_lot", "portable-atomic", "race", "std", "unstable"))' -C metadata=6870d82906744d65 -C extra-filename=-6870d82906744d65 --out-dir /home/trent/src/tiktoken/target/release/deps -C linker=/home/trent/mambaforge/envs/py313t/bin/x86_64-conda-linux-gnu-cc -C strip=debuginfo -L dependency=/home/trent/src/tiktoken/target/release/deps --cap-lints allow`
     Running `rustc --crate-name build_script_build --edition=2021 /home/trent/.cargo/registry/src/index.crates.io-6f17d22bba15001f/proc-macro2-1.0.92/build.rs --error-format=json --json=diagnostic-rendered-ansi,artifacts,future-incompat --diagnostic-width=254 --crate-type bin --emit=dep-info,link -C embed-bitcode=no -C debug-assertions=off --cfg 'feature="proc-macro"' --check-cfg 'cfg(docsrs)' --check-cfg 'cfg(feature, values("default", "nightly", "proc-macro", "span-locations"))' -C metadata=4463b02a2f05d75c -C extra-filename=-4463b02a2f05d75c --out-dir /home/trent/src/tiktoken/target/release/build/proc-macro2-4463b02a2f05d75c -C linker=/home/trent/mambaforge/envs/py313t/bin/x86_64-conda-linux-gnu-cc -C strip=debuginfo -L dependency=/home/trent/src/tiktoken/target/release/deps --cap-lints allow`
     Running `rustc --crate-name unicode_ident --edition=2018 /home/trent/.cargo/registry/src/index.crates.io-6f17d22bba15001f/unicode-ident-1.0.14/src/lib.rs --error-format=json --json=diagnostic-rendered-ansi,artifacts,future-incompat --diagnostic-width=254 --crate-type lib --emit=dep-info,metadata,link -C embed-bitcode=no -C debug-assertions=off --check-cfg 'cfg(docsrs)' --check-cfg 'cfg(feature, values())' -C metadata=17278dafa5f26de9 -C extra-filename=-17278dafa5f26de9 --out-dir /home/trent/src/tiktoken/target/release/deps -C linker=/home/trent/mambaforge/envs/py313t/bin/x86_64-conda-linux-gnu-cc -C strip=debuginfo -L dependency=/home/trent/src/tiktoken/target/release/deps --cap-lints allow`
     Running `rustc --crate-name memchr --edition=2021 /home/trent/.cargo/registry/src/index.crates.io-6f17d22bba15001f/memchr-2.7.4/src/lib.rs --error-format=json --json=diagnostic-rendered-ansi,artifacts,future-incompat --diagnostic-width=254 --crate-type lib --emit=dep-info,metadata,link -C opt-level=3 -C embed-bitcode=no --cfg 'feature="alloc"' --cfg 'feature="std"' --check-cfg 'cfg(docsrs)' --check-cfg 'cfg(feature, values("alloc", "compiler_builtins", "core", "default", "libc", "logging", "rustc-dep-of-std", "std", "use_std"))' -C metadata=25fa9792dd9399b0 -C extra-filename=-25fa9792dd9399b0 --out-dir /home/trent/src/tiktoken/target/release/deps -C linker=/home/trent/mambaforge/envs/py313t/bin/x86_64-conda-linux-gnu-cc -C strip=debuginfo -L dependency=/home/trent/src/tiktoken/target/release/deps --cap-lints allow`
     Running `rustc --crate-name regex_syntax --edition=2021 /home/trent/.cargo/registry/src/index.crates.io-6f17d22bba15001f/regex-syntax-0.8.5/src/lib.rs --error-format=json --json=diagnostic-rendered-ansi,artifacts,future-incompat --diagnostic-width=254 --crate-type lib --emit=dep-info,metadata,link -C opt-level=3 -C embed-bitcode=no --cfg 'feature="default"' --cfg 'feature="std"' --cfg 'feature="unicode"' --cfg 'feature="unicode-age"' --cfg 'feature="unicode-bool"' --cfg 'feature="unicode-case"' --cfg 'feature="unicode-gencat"' --cfg 'feature="unicode-perl"' --cfg 'feature="unicode-script"' --cfg 'feature="unicode-segment"' --check-cfg 'cfg(docsrs)' --check-cfg 'cfg(feature, values("arbitrary", "default", "std", "unicode", "unicode-age", "unicode-bool", "unicode-case", "unicode-gencat", "unicode-perl", "unicode-script", "unicode-segment"))' -C metadata=66f570e05dbe3825 -C extra-filename=-66f570e05dbe3825 --out-dir /home/trent/src/tiktoken/target/release/deps -C linker=/home/trent/mambaforge/envs/py313t/bin/x86_64-conda-linux-gnu-cc -C strip=debuginfo -L dependency=/home/trent/src/tiktoken/target/release/deps --cap-lints allow`
     Running `rustc --crate-name build_script_build --edition=2021 /home/trent/.cargo/registry/src/index.crates.io-6f17d22bba15001f/libc-0.2.169/build.rs --error-format=json --json=diagnostic-rendered-ansi,artifacts,future-incompat --diagnostic-width=254 --crate-type bin --emit=dep-info,link -C embed-bitcode=no -C debug-assertions=off --cfg 'feature="default"' --cfg 'feature="std"' --check-cfg 'cfg(docsrs)' --check-cfg 'cfg(feature, values("align", "const-extern-fn", "default", "extra_traits", "rustc-dep-of-std", "rustc-std-workspace-core", "std", "use_std"))' -C metadata=42fd5088387abf7a -C extra-filename=-42fd5088387abf7a --out-dir /home/trent/src/tiktoken/target/release/build/libc-42fd5088387abf7a -C linker=/home/trent/mambaforge/envs/py313t/bin/x86_64-conda-linux-gnu-cc -C strip=debuginfo -L dependency=/home/trent/src/tiktoken/target/release/deps --cap-lints allow`
     Running `rustc --crate-name autocfg --edition=2015 /home/trent/.cargo/registry/src/index.crates.io-6f17d22bba15001f/autocfg-1.4.0/src/lib.rs --error-format=json --json=diagnostic-rendered-ansi,artifacts,future-incompat --diagnostic-width=254 --crate-type lib --emit=dep-info,metadata,link -C embed-bitcode=no -C debug-assertions=off --check-cfg 'cfg(docsrs)' --check-cfg 'cfg(feature, values())' -C metadata=b8e4c5d316ce5bfb -C extra-filename=-b8e4c5d316ce5bfb --out-dir /home/trent/src/tiktoken/target/release/deps -C linker=/home/trent/mambaforge/envs/py313t/bin/x86_64-conda-linux-gnu-cc -C strip=debuginfo -L dependency=/home/trent/src/tiktoken/target/release/deps --cap-lints allow`
     Running `rustc --crate-name bit_vec --edition=2015 /home/trent/.cargo/registry/src/index.crates.io-6f17d22bba15001f/bit-vec-0.6.3/src/lib.rs --error-format=json --json=diagnostic-rendered-ansi,artifacts,future-incompat --diagnostic-width=254 --crate-type lib --emit=dep-info,metadata,link -C opt-level=3 -C embed-bitcode=no --cfg 'feature="std"' --check-cfg 'cfg(docsrs)' --check-cfg 'cfg(feature, values("default", "serde", "serde_no_std", "serde_std", "std"))' -C metadata=de2a0d1e2ef2490a -C extra-filename=-de2a0d1e2ef2490a --out-dir /home/trent/src/tiktoken/target/release/deps -C linker=/home/trent/mambaforge/envs/py313t/bin/x86_64-conda-linux-gnu-cc -C strip=debuginfo -L dependency=/home/trent/src/tiktoken/target/release/deps --cap-lints allow`
     Running `rustc --crate-name heck --edition=2021 /home/trent/.cargo/registry/src/index.crates.io-6f17d22bba15001f/heck-0.5.0/src/lib.rs --error-format=json --json=diagnostic-rendered-ansi,artifacts,future-incompat --diagnostic-width=254 --crate-type lib --emit=dep-info,metadata,link -C embed-bitcode=no -C debug-assertions=off --check-cfg 'cfg(docsrs)' --check-cfg 'cfg(feature, values())' -C metadata=5e22b1dffa7f4255 -C extra-filename=-5e22b1dffa7f4255 --out-dir /home/trent/src/tiktoken/target/release/deps -C linker=/home/trent/mambaforge/envs/py313t/bin/x86_64-conda-linux-gnu-cc -C strip=debuginfo -L dependency=/home/trent/src/tiktoken/target/release/deps --cap-lints allow`
     Running `rustc --crate-name once_cell --edition=2021 /home/trent/.cargo/registry/src/index.crates.io-6f17d22bba15001f/once_cell-1.20.2/src/lib.rs --error-format=json --json=diagnostic-rendered-ansi,artifacts,future-incompat --diagnostic-width=254 --crate-type lib --emit=dep-info,metadata,link -C opt-level=3 -C embed-bitcode=no --cfg 'feature="alloc"' --cfg 'feature="default"' --cfg 'feature="race"' --cfg 'feature="std"' --check-cfg 'cfg(docsrs)' --check-cfg 'cfg(feature, values("alloc", "atomic-polyfill", "critical-section", "default", "parking_lot", "portable-atomic", "race", "std", "unstable"))' -C metadata=05003007543cfa87 -C extra-filename=-05003007543cfa87 --out-dir /home/trent/src/tiktoken/target/release/deps -C linker=/home/trent/mambaforge/envs/py313t/bin/x86_64-conda-linux-gnu-cc -C strip=debuginfo -L dependency=/home/trent/src/tiktoken/target/release/deps --cap-lints allow`
     Running `rustc --crate-name unindent --edition=2021 /home/trent/.cargo/registry/src/index.crates.io-6f17d22bba15001f/unindent-0.2.3/src/lib.rs --error-format=json --json=diagnostic-rendered-ansi,artifacts,future-incompat --diagnostic-width=254 --crate-type lib --emit=dep-info,metadata,link -C opt-level=3 -C embed-bitcode=no --check-cfg 'cfg(docsrs)' --check-cfg 'cfg(feature, values())' -C metadata=d53e2a0385a47a80 -C extra-filename=-d53e2a0385a47a80 --out-dir /home/trent/src/tiktoken/target/release/deps -C linker=/home/trent/mambaforge/envs/py313t/bin/x86_64-conda-linux-gnu-cc -C strip=debuginfo -L dependency=/home/trent/src/tiktoken/target/release/deps --cap-lints allow`
     Running `rustc --crate-name indoc --edition=2021 /home/trent/.cargo/registry/src/index.crates.io-6f17d22bba15001f/indoc-2.0.5/src/lib.rs --error-format=json --json=diagnostic-rendered-ansi,artifacts,future-incompat --diagnostic-width=254 --crate-type proc-macro --emit=dep-info,link -C prefer-dynamic -C embed-bitcode=no -C debug-assertions=off --check-cfg 'cfg(docsrs)' --check-cfg 'cfg(feature, values())' -C metadata=b4e94d9ecbd21a39 -C extra-filename=-b4e94d9ecbd21a39 --out-dir /home/trent/src/tiktoken/target/release/deps -C linker=/home/trent/mambaforge/envs/py313t/bin/x86_64-conda-linux-gnu-cc -C strip=debuginfo -L dependency=/home/trent/src/tiktoken/target/release/deps --extern proc_macro --cap-lints allow`
     Running `rustc --crate-name cfg_if --edition=2018 /home/trent/.cargo/registry/src/index.crates.io-6f17d22bba15001f/cfg-if-1.0.0/src/lib.rs --error-format=json --json=diagnostic-rendered-ansi,artifacts,future-incompat --diagnostic-width=254 --crate-type lib --emit=dep-info,metadata,link -C opt-level=3 -C embed-bitcode=no --check-cfg 'cfg(docsrs)' --check-cfg 'cfg(feature, values("compiler_builtins", "core", "rustc-dep-of-std"))' -C metadata=e4ded2c19830fbdd -C extra-filename=-e4ded2c19830fbdd --out-dir /home/trent/src/tiktoken/target/release/deps -C linker=/home/trent/mambaforge/envs/py313t/bin/x86_64-conda-linux-gnu-cc -C strip=debuginfo -L dependency=/home/trent/src/tiktoken/target/release/deps --cap-lints allow`
     Running `rustc --crate-name rustc_hash --edition=2015 /home/trent/.cargo/registry/src/index.crates.io-6f17d22bba15001f/rustc-hash-1.1.0/src/lib.rs --error-format=json --json=diagnostic-rendered-ansi,artifacts,future-incompat --diagnostic-width=254 --crate-type lib --emit=dep-info,metadata,link -C opt-level=3 -C embed-bitcode=no --cfg 'feature="default"' --cfg 'feature="std"' --check-cfg 'cfg(docsrs)' --check-cfg 'cfg(feature, values("default", "std"))' -C metadata=b006f4d81e95dfaf -C extra-filename=-b006f4d81e95dfaf --out-dir /home/trent/src/tiktoken/target/release/deps -C linker=/home/trent/mambaforge/envs/py313t/bin/x86_64-conda-linux-gnu-cc -C strip=debuginfo -L dependency=/home/trent/src/tiktoken/target/release/deps --cap-lints allow`
   Compiling bit-set v0.5.3
     Running `rustc --crate-name bit_set --edition=2015 /home/trent/.cargo/registry/src/index.crates.io-6f17d22bba15001f/bit-set-0.5.3/src/lib.rs --error-format=json --json=diagnostic-rendered-ansi,artifacts,future-incompat --diagnostic-width=254 --crate-type lib --emit=dep-info,metadata,link -C opt-level=3 -C embed-bitcode=no --cfg 'feature="std"' --check-cfg 'cfg(docsrs)' --check-cfg 'cfg(feature, values("default", "std"))' -C metadata=40d90f83eb5bab57 -C extra-filename=-40d90f83eb5bab57 --out-dir /home/trent/src/tiktoken/target/release/deps -C linker=/home/trent/mambaforge/envs/py313t/bin/x86_64-conda-linux-gnu-cc -C strip=debuginfo -L dependency=/home/trent/src/tiktoken/target/release/deps --extern bit_vec=/home/trent/src/tiktoken/target/release/deps/libbit_vec-de2a0d1e2ef2490a.rmeta --cap-lints allow`
     Running `/home/trent/src/tiktoken/target/release/build/libc-42fd5088387abf7a/build-script-build`
     Running `/home/trent/src/tiktoken/target/release/build/proc-macro2-4463b02a2f05d75c/build-script-build`
   Compiling memoffset v0.9.1
     Running `rustc --crate-name build_script_build --edition=2015 /home/trent/.cargo/registry/src/index.crates.io-6f17d22bba15001f/memoffset-0.9.1/build.rs --error-format=json --json=diagnostic-rendered-ansi,artifacts,future-incompat --diagnostic-width=254 --crate-type bin --emit=dep-info,link -C embed-bitcode=no -C debug-assertions=off --cfg 'feature="default"' --check-cfg 'cfg(docsrs)' --check-cfg 'cfg(feature, values("default", "unstable_const", "unstable_offset_of"))' -C metadata=679ebbc3261d6845 -C extra-filename=-679ebbc3261d6845 --out-dir /home/trent/src/tiktoken/target/release/build/memoffset-679ebbc3261d6845 -C linker=/home/trent/mambaforge/envs/py313t/bin/x86_64-conda-linux-gnu-cc -C strip=debuginfo -L dependency=/home/trent/src/tiktoken/target/release/deps --extern autocfg=/home/trent/src/tiktoken/target/release/deps/libautocfg-b8e4c5d316ce5bfb.rlib --cap-lints allow`
     Running `rustc --crate-name libc --edition=2021 /home/trent/.cargo/registry/src/index.crates.io-6f17d22bba15001f/libc-0.2.169/src/lib.rs --error-format=json --json=diagnostic-rendered-ansi,artifacts,future-incompat --diagnostic-width=254 --crate-type lib --emit=dep-info,metadata,link -C opt-level=3 -C embed-bitcode=no --cfg 'feature="default"' --cfg 'feature="std"' --check-cfg 'cfg(docsrs)' --check-cfg 'cfg(feature, values("align", "const-extern-fn", "default", "extra_traits", "rustc-dep-of-std", "rustc-std-workspace-core", "std", "use_std"))' -C metadata=6b0fbe5bd5ba9d30 -C extra-filename=-6b0fbe5bd5ba9d30 --out-dir /home/trent/src/tiktoken/target/release/deps -C linker=/home/trent/mambaforge/envs/py313t/bin/x86_64-conda-linux-gnu-cc -C strip=debuginfo -L dependency=/home/trent/src/tiktoken/target/release/deps --cap-lints allow --cfg freebsd11 --cfg libc_const_extern_fn --check-cfg 'cfg(emscripten_new_stat_abi)' --check-cfg 'cfg(espidf_time32)' --check-cfg 'cfg(freebsd10)' --check-cfg 'cfg(freebsd11)' --check-cfg 'cfg(freebsd12)' --check-cfg 'cfg(freebsd13)' --check-cfg 'cfg(freebsd14)' --check-cfg 'cfg(freebsd15)' --check-cfg 'cfg(libc_const_extern_fn)' --check-cfg 'cfg(libc_deny_warnings)' --check-cfg 'cfg(libc_thread_local)' --check-cfg 'cfg(libc_ctest)' --check-cfg 'cfg(target_os,values("switch","aix","ohos","hurd","rtems","visionos","nuttx"))' --check-cfg 'cfg(target_env,values("illumos","wasi","aix","ohos"))' --check-cfg 'cfg(target_arch,values("loongarch64","mips32r6","mips64r6","csky"))'`
     Running `rustc --crate-name proc_macro2 --edition=2021 /home/trent/.cargo/registry/src/index.crates.io-6f17d22bba15001f/proc-macro2-1.0.92/src/lib.rs --error-format=json --json=diagnostic-rendered-ansi,artifacts,future-incompat --diagnostic-width=254 --crate-type lib --emit=dep-info,metadata,link -C embed-bitcode=no -C debug-assertions=off --cfg 'feature="proc-macro"' --check-cfg 'cfg(docsrs)' --check-cfg 'cfg(feature, values("default", "nightly", "proc-macro", "span-locations"))' -C metadata=4c69c42d9df03375 -C extra-filename=-4c69c42d9df03375 --out-dir /home/trent/src/tiktoken/target/release/deps -C linker=/home/trent/mambaforge/envs/py313t/bin/x86_64-conda-linux-gnu-cc -C strip=debuginfo -L dependency=/home/trent/src/tiktoken/target/release/deps --extern unicode_ident=/home/trent/src/tiktoken/target/release/deps/libunicode_ident-17278dafa5f26de9.rmeta --cap-lints allow --cfg wrap_proc_macro --check-cfg 'cfg(fuzzing)' --check-cfg 'cfg(no_is_available)' --check-cfg 'cfg(no_literal_byte_character)' --check-cfg 'cfg(no_literal_c_string)' --check-cfg 'cfg(no_source_text)' --check-cfg 'cfg(proc_macro_span)' --check-cfg 'cfg(procmacro2_backtrace)' --check-cfg 'cfg(procmacro2_nightly_testing)' --check-cfg 'cfg(procmacro2_semver_exempt)' --check-cfg 'cfg(randomize_layout)' --check-cfg 'cfg(span_locations)' --check-cfg 'cfg(super_unstable)' --check-cfg 'cfg(wrap_proc_macro)'`
     Running `/home/trent/src/tiktoken/target/release/build/target-lexicon-826df8be4fa9ef21/build-script-build`
     Running `rustc --crate-name target_lexicon --edition=2018 /home/trent/.cargo/registry/src/index.crates.io-6f17d22bba15001f/target-lexicon-0.12.16/src/lib.rs --error-format=json --json=diagnostic-rendered-ansi,artifacts,future-incompat --diagnostic-width=254 --crate-type lib --emit=dep-info,metadata,link -C embed-bitcode=no -C debug-assertions=off --cfg 'feature="default"' --check-cfg 'cfg(docsrs)' --check-cfg 'cfg(feature, values("arch_zkasm", "default", "serde", "serde_support", "std"))' -C metadata=a879275207ec599a -C extra-filename=-a879275207ec599a --out-dir /home/trent/src/tiktoken/target/release/deps -C linker=/home/trent/mambaforge/envs/py313t/bin/x86_64-conda-linux-gnu-cc -C strip=debuginfo -L dependency=/home/trent/src/tiktoken/target/release/deps --cap-lints allow --cfg 'feature="rust_1_40"'`
     Running `/home/trent/src/tiktoken/target/release/build/memoffset-679ebbc3261d6845/build-script-build`
     Running `rustc --crate-name memoffset --edition=2015 /home/trent/.cargo/registry/src/index.crates.io-6f17d22bba15001f/memoffset-0.9.1/src/lib.rs --error-format=json --json=diagnostic-rendered-ansi,artifacts,future-incompat --diagnostic-width=254 --crate-type lib --emit=dep-info,metadata,link -C opt-level=3 -C embed-bitcode=no --cfg 'feature="default"' --check-cfg 'cfg(docsrs)' --check-cfg 'cfg(feature, values("default", "unstable_const", "unstable_offset_of"))' -C metadata=65fe1a8a113b3005 -C extra-filename=-65fe1a8a113b3005 --out-dir /home/trent/src/tiktoken/target/release/deps -C linker=/home/trent/mambaforge/envs/py313t/bin/x86_64-conda-linux-gnu-cc -C strip=debuginfo -L dependency=/home/trent/src/tiktoken/target/release/deps --cap-lints allow --cfg tuple_ty --cfg allow_clippy --cfg maybe_uninit --cfg doctests --cfg raw_ref_macros --cfg stable_const --cfg stable_offset_of`
   Compiling aho-corasick v1.1.3
     Running `rustc --crate-name aho_corasick --edition=2021 /home/trent/.cargo/registry/src/index.crates.io-6f17d22bba15001f/aho-corasick-1.1.3/src/lib.rs --error-format=json --json=diagnostic-rendered-ansi,artifacts,future-incompat --diagnostic-width=254 --crate-type lib --emit=dep-info,metadata,link -C opt-level=3 -C embed-bitcode=no --cfg 'feature="perf-literal"' --cfg 'feature="std"' --check-cfg 'cfg(docsrs)' --check-cfg 'cfg(feature, values("default", "logging", "perf-literal", "std"))' -C metadata=6668c9f838fb89b3 -C extra-filename=-6668c9f838fb89b3 --out-dir /home/trent/src/tiktoken/target/release/deps -C linker=/home/trent/mambaforge/envs/py313t/bin/x86_64-conda-linux-gnu-cc -C strip=debuginfo -L dependency=/home/trent/src/tiktoken/target/release/deps --extern memchr=/home/trent/src/tiktoken/target/release/deps/libmemchr-25fa9792dd9399b0.rmeta --cap-lints allow`
   Compiling pyo3-build-config v0.23.3
     Running `rustc --crate-name build_script_build --edition=2021 /home/trent/.cargo/registry/src/index.crates.io-6f17d22bba15001f/pyo3-build-config-0.23.3/build.rs --error-format=json --json=diagnostic-rendered-ansi,artifacts,future-incompat --diagnostic-width=254 --crate-type bin --emit=dep-info,link -C embed-bitcode=no -C debug-assertions=off --cfg 'feature="default"' --cfg 'feature="extension-module"' --cfg 'feature="resolve-config"' --check-cfg 'cfg(docsrs)' --check-cfg 'cfg(feature, values("abi3", "abi3-py310", "abi3-py311", "abi3-py312", "abi3-py37", "abi3-py38", "abi3-py39", "default", "extension-module", "python3-dll-a", "resolve-config"))' -C metadata=d8ab86bb094cf645 -C extra-filename=-d8ab86bb094cf645 --out-dir /home/trent/src/tiktoken/target/release/build/pyo3-build-config-d8ab86bb094cf645 -C linker=/home/trent/mambaforge/envs/py313t/bin/x86_64-conda-linux-gnu-cc -C strip=debuginfo -L dependency=/home/trent/src/tiktoken/target/release/deps --extern target_lexicon=/home/trent/src/tiktoken/target/release/deps/libtarget_lexicon-a879275207ec599a.rlib --cap-lints allow`
   Compiling quote v1.0.38
     Running `rustc --crate-name quote --edition=2018 /home/trent/.cargo/registry/src/index.crates.io-6f17d22bba15001f/quote-1.0.38/src/lib.rs --error-format=json --json=diagnostic-rendered-ansi,artifacts,future-incompat --diagnostic-width=254 --crate-type lib --emit=dep-info,metadata,link -C embed-bitcode=no -C debug-assertions=off --cfg 'feature="default"' --cfg 'feature="proc-macro"' --check-cfg 'cfg(docsrs)' --check-cfg 'cfg(feature, values("default", "proc-macro"))' -C metadata=169332b6fe3d21d6 -C extra-filename=-169332b6fe3d21d6 --out-dir /home/trent/src/tiktoken/target/release/deps -C linker=/home/trent/mambaforge/envs/py313t/bin/x86_64-conda-linux-gnu-cc -C strip=debuginfo -L dependency=/home/trent/src/tiktoken/target/release/deps --extern proc_macro2=/home/trent/src/tiktoken/target/release/deps/libproc_macro2-4c69c42d9df03375.rmeta --cap-lints allow`
   Compiling syn v2.0.95
     Running `rustc --crate-name syn --edition=2021 /home/trent/.cargo/registry/src/index.crates.io-6f17d22bba15001f/syn-2.0.95/src/lib.rs --error-format=json --json=diagnostic-rendered-ansi,artifacts,future-incompat --diagnostic-width=254 --crate-type lib --emit=dep-info,metadata,link -C embed-bitcode=no -C debug-assertions=off --cfg 'feature="clone-impls"' --cfg 'feature="default"' --cfg 'feature="derive"' --cfg 'feature="extra-traits"' --cfg 'feature="full"' --cfg 'feature="parsing"' --cfg 'feature="printing"' --cfg 'feature="proc-macro"' --check-cfg 'cfg(docsrs)' --check-cfg 'cfg(feature, values("clone-impls", "default", "derive", "extra-traits", "fold", "full", "parsing", "printing", "proc-macro", "test", "visit", "visit-mut"))' -C metadata=29d4a0ddbd98f61f -C extra-filename=-29d4a0ddbd98f61f --out-dir /home/trent/src/tiktoken/target/release/deps -C linker=/home/trent/mambaforge/envs/py313t/bin/x86_64-conda-linux-gnu-cc -C strip=debuginfo -L dependency=/home/trent/src/tiktoken/target/release/deps --extern proc_macro2=/home/trent/src/tiktoken/target/release/deps/libproc_macro2-4c69c42d9df03375.rmeta --extern quote=/home/trent/src/tiktoken/target/release/deps/libquote-169332b6fe3d21d6.rmeta --extern unicode_ident=/home/trent/src/tiktoken/target/release/deps/libunicode_ident-17278dafa5f26de9.rmeta --cap-lints allow`
     Running `/home/trent/src/tiktoken/target/release/build/pyo3-build-config-d8ab86bb094cf645/build-script-build`
     Running `rustc --crate-name pyo3_build_config --edition=2021 /home/trent/.cargo/registry/src/index.crates.io-6f17d22bba15001f/pyo3-build-config-0.23.3/src/lib.rs --error-format=json --json=diagnostic-rendered-ansi,artifacts,future-incompat --diagnostic-width=254 --crate-type lib --emit=dep-info,metadata,link -C embed-bitcode=no -C debug-assertions=off --cfg 'feature="default"' --cfg 'feature="extension-module"' --cfg 'feature="resolve-config"' --check-cfg 'cfg(docsrs)' --check-cfg 'cfg(feature, values("abi3", "abi3-py310", "abi3-py311", "abi3-py312", "abi3-py37", "abi3-py38", "abi3-py39", "default", "extension-module", "python3-dll-a", "resolve-config"))' -C metadata=b884267014f5753b -C extra-filename=-b884267014f5753b --out-dir /home/trent/src/tiktoken/target/release/deps -C linker=/home/trent/mambaforge/envs/py313t/bin/x86_64-conda-linux-gnu-cc -C strip=debuginfo -L dependency=/home/trent/src/tiktoken/target/release/deps --extern once_cell=/home/trent/src/tiktoken/target/release/deps/libonce_cell-6870d82906744d65.rmeta --extern target_lexicon=/home/trent/src/tiktoken/target/release/deps/libtarget_lexicon-a879275207ec599a.rmeta --cap-lints allow`
   Compiling pyo3-ffi v0.23.3
   Compiling pyo3-macros-backend v0.23.3
   Compiling pyo3 v0.23.3
     Running `rustc --crate-name build_script_build --edition=2021 /home/trent/.cargo/registry/src/index.crates.io-6f17d22bba15001f/pyo3-ffi-0.23.3/build.rs --error-format=json --json=diagnostic-rendered-ansi,artifacts,future-incompat --diagnostic-width=254 --crate-type bin --emit=dep-info,link -C embed-bitcode=no --warn=rust_2018_idioms '--warn=clippy::useless_transmute' '--warn=clippy::used_underscore_binding' --warn=unused_lifetimes '--warn=clippy::unnecessary_wraps' '--warn=clippy::todo' --warn=rust_2021_prelude_collisions '--warn=clippy::manual_ok_or' '--warn=clippy::manual_assert' '--warn=clippy::let_unit_value' --warn=invalid_doc_attributes '--warn=clippy::flat_map_option' '--warn=clippy::filter_map_next' '--warn=clippy::explicit_iter_loop' '--warn=clippy::explicit_into_iter_loop' --warn=elided_lifetimes_in_paths '--warn=clippy::dbg_macro' '--warn=clippy::checked_conversions' '--warn=rustdoc::broken_intra_doc_links' '--warn=rustdoc::bare_urls' -C debug-assertions=off --cfg 'feature="default"' --cfg 'feature="extension-module"' --check-cfg 'cfg(docsrs)' --check-cfg 'cfg(feature, values("abi3", "abi3-py310", "abi3-py311", "abi3-py312", "abi3-py37", "abi3-py38", "abi3-py39", "default", "extension-module", "generate-import-lib"))' -C metadata=5c5f8f108a22b6ae -C extra-filename=-5c5f8f108a22b6ae --out-dir /home/trent/src/tiktoken/target/release/build/pyo3-ffi-5c5f8f108a22b6ae -C linker=/home/trent/mambaforge/envs/py313t/bin/x86_64-conda-linux-gnu-cc -C strip=debuginfo -L dependency=/home/trent/src/tiktoken/target/release/deps --extern pyo3_build_config=/home/trent/src/tiktoken/target/release/deps/libpyo3_build_config-b884267014f5753b.rlib --cap-lints allow`
     Running `rustc --crate-name build_script_build --edition=2021 /home/trent/.cargo/registry/src/index.crates.io-6f17d22bba15001f/pyo3-macros-backend-0.23.3/build.rs --error-format=json --json=diagnostic-rendered-ansi,artifacts,future-incompat --diagnostic-width=254 --crate-type bin --emit=dep-info,link -C embed-bitcode=no --warn=rust_2018_idioms '--warn=clippy::useless_transmute' '--warn=clippy::used_underscore_binding' --warn=unused_lifetimes '--warn=clippy::unnecessary_wraps' '--warn=clippy::todo' --warn=rust_2021_prelude_collisions '--warn=clippy::manual_ok_or' '--warn=clippy::manual_assert' '--warn=clippy::let_unit_value' --warn=invalid_doc_attributes '--warn=clippy::flat_map_option' '--warn=clippy::filter_map_next' '--warn=clippy::explicit_iter_loop' '--warn=clippy::explicit_into_iter_loop' --warn=elided_lifetimes_in_paths '--warn=clippy::dbg_macro' '--warn=clippy::checked_conversions' '--warn=rustdoc::broken_intra_doc_links' '--warn=rustdoc::bare_urls' -C debug-assertions=off --check-cfg 'cfg(docsrs)' --check-cfg 'cfg(feature, values("experimental-async"))' -C metadata=bb4a4b3911c85e51 -C extra-filename=-bb4a4b3911c85e51 --out-dir /home/trent/src/tiktoken/target/release/build/pyo3-macros-backend-bb4a4b3911c85e51 -C linker=/home/trent/mambaforge/envs/py313t/bin/x86_64-conda-linux-gnu-cc -C strip=debuginfo -L dependency=/home/trent/src/tiktoken/target/release/deps --extern pyo3_build_config=/home/trent/src/tiktoken/target/release/deps/libpyo3_build_config-b884267014f5753b.rlib --cap-lints allow`
     Running `rustc --crate-name build_script_build --edition=2021 /home/trent/.cargo/registry/src/index.crates.io-6f17d22bba15001f/pyo3-0.23.3/build.rs --error-format=json --json=diagnostic-rendered-ansi,artifacts,future-incompat --diagnostic-width=254 --crate-type bin --emit=dep-info,link -C embed-bitcode=no --warn=rust_2018_idioms '--warn=clippy::useless_transmute' '--warn=clippy::used_underscore_binding' --warn=unused_lifetimes '--warn=clippy::unnecessary_wraps' '--warn=clippy::todo' --warn=rust_2021_prelude_collisions '--warn=clippy::manual_ok_or' '--warn=clippy::manual_assert' '--warn=clippy::let_unit_value' --warn=invalid_doc_attributes '--warn=clippy::flat_map_option' '--warn=clippy::filter_map_next' '--warn=clippy::explicit_iter_loop' '--warn=clippy::explicit_into_iter_loop' --warn=elided_lifetimes_in_paths '--warn=clippy::dbg_macro' '--warn=clippy::checked_conversions' '--warn=rustdoc::broken_intra_doc_links' '--warn=rustdoc::bare_urls' -C debug-assertions=off --cfg 'feature="extension-module"' --cfg 'feature="indoc"' --cfg 'feature="macros"' --cfg 'feature="pyo3-macros"' --cfg 'feature="unindent"' --check-cfg 'cfg(docsrs)' --check-cfg 'cfg(feature, values("abi3", "abi3-py310", "abi3-py311", "abi3-py312", "abi3-py37", "abi3-py38", "abi3-py39", "anyhow", "auto-initialize", "chrono", "chrono-tz", "default", "either", "experimental-async", "experimental-inspect", "extension-module", "eyre", "full", "generate-import-lib", "hashbrown", "indexmap", "indoc", "inventory", "macros", "multiple-pymethods", "nightly", "num-bigint", "num-complex", "num-rational", "py-clone", "pyo3-macros", "rust_decimal", "serde", "smallvec", "unindent"))' -C metadata=65f0dbfaaf67ed5a -C extra-filename=-65f0dbfaaf67ed5a --out-dir /home/trent/src/tiktoken/target/release/build/pyo3-65f0dbfaaf67ed5a -C linker=/home/trent/mambaforge/envs/py313t/bin/x86_64-conda-linux-gnu-cc -C strip=debuginfo -L dependency=/home/trent/src/tiktoken/target/release/deps --extern pyo3_build_config=/home/trent/src/tiktoken/target/release/deps/libpyo3_build_config-b884267014f5753b.rlib --cap-lints allow`
   Compiling regex-automata v0.4.9
     Running `rustc --crate-name regex_automata --edition=2021 /home/trent/.cargo/registry/src/index.crates.io-6f17d22bba15001f/regex-automata-0.4.9/src/lib.rs --error-format=json --json=diagnostic-rendered-ansi,artifacts,future-incompat --diagnostic-width=254 --crate-type lib --emit=dep-info,metadata,link -C opt-level=3 -C embed-bitcode=no --cfg 'feature="alloc"' --cfg 'feature="dfa"' --cfg 'feature="dfa-build"' --cfg 'feature="dfa-onepass"' --cfg 'feature="dfa-search"' --cfg 'feature="hybrid"' --cfg 'feature="meta"' --cfg 'feature="nfa"' --cfg 'feature="nfa-backtrack"' --cfg 'feature="nfa-pikevm"' --cfg 'feature="nfa-thompson"' --cfg 'feature="perf"' --cfg 'feature="perf-inline"' --cfg 'feature="perf-literal"' --cfg 'feature="perf-literal-multisubstring"' --cfg 'feature="perf-literal-substring"' --cfg 'feature="std"' --cfg 'feature="syntax"' --cfg 'feature="unicode"' --cfg 'feature="unicode-age"' --cfg 'feature="unicode-bool"' --cfg 'feature="unicode-case"' --cfg 'feature="unicode-gencat"' --cfg 'feature="unicode-perl"' --cfg 'feature="unicode-script"' --cfg 'feature="unicode-segment"' --cfg 'feature="unicode-word-boundary"' --check-cfg 'cfg(docsrs)' --check-cfg 'cfg(feature, values("alloc", "default", "dfa", "dfa-build", "dfa-onepass", "dfa-search", "hybrid", "internal-instrument", "internal-instrument-pikevm", "logging", "meta", "nfa", "nfa-backtrack", "nfa-pikevm", "nfa-thompson", "perf", "perf-inline", "perf-literal", "perf-literal-multisubstring", "perf-literal-substring", "std", "syntax", "unicode", "unicode-age", "unicode-bool", "unicode-case", "unicode-gencat", "unicode-perl", "unicode-script", "unicode-segment", "unicode-word-boundary"))' -C metadata=01135b6ed6d4413e -C extra-filename=-01135b6ed6d4413e --out-dir /home/trent/src/tiktoken/target/release/deps -C linker=/home/trent/mambaforge/envs/py313t/bin/x86_64-conda-linux-gnu-cc -C strip=debuginfo -L dependency=/home/trent/src/tiktoken/target/release/deps --extern aho_corasick=/home/trent/src/tiktoken/target/release/deps/libaho_corasick-6668c9f838fb89b3.rmeta --extern memchr=/home/trent/src/tiktoken/target/release/deps/libmemchr-25fa9792dd9399b0.rmeta --extern regex_syntax=/home/trent/src/tiktoken/target/release/deps/libregex_syntax-66f570e05dbe3825.rmeta --cap-lints allow`
     Running `/home/trent/src/tiktoken/target/release/build/pyo3-macros-backend-bb4a4b3911c85e51/build-script-build`
     Running `/home/trent/src/tiktoken/target/release/build/pyo3-ffi-5c5f8f108a22b6ae/build-script-build`
     Running `/home/trent/src/tiktoken/target/release/build/pyo3-65f0dbfaaf67ed5a/build-script-build`
     Running `rustc --crate-name pyo3_ffi --edition=2021 /home/trent/.cargo/registry/src/index.crates.io-6f17d22bba15001f/pyo3-ffi-0.23.3/src/lib.rs --error-format=json --json=diagnostic-rendered-ansi,artifacts,future-incompat --diagnostic-width=254 --crate-type lib --emit=dep-info,metadata,link -C opt-level=3 -C embed-bitcode=no --warn=rust_2018_idioms '--warn=clippy::useless_transmute' '--warn=clippy::used_underscore_binding' --warn=unused_lifetimes '--warn=clippy::unnecessary_wraps' '--warn=clippy::todo' --warn=rust_2021_prelude_collisions '--warn=clippy::manual_ok_or' '--warn=clippy::manual_assert' '--warn=clippy::let_unit_value' --warn=invalid_doc_attributes '--warn=clippy::flat_map_option' '--warn=clippy::filter_map_next' '--warn=clippy::explicit_iter_loop' '--warn=clippy::explicit_into_iter_loop' --warn=elided_lifetimes_in_paths '--warn=clippy::dbg_macro' '--warn=clippy::checked_conversions' '--warn=rustdoc::broken_intra_doc_links' '--warn=rustdoc::bare_urls' --cfg 'feature="default"' --cfg 'feature="extension-module"' --check-cfg 'cfg(docsrs)' --check-cfg 'cfg(feature, values("abi3", "abi3-py310", "abi3-py311", "abi3-py312", "abi3-py37", "abi3-py38", "abi3-py39", "default", "extension-module", "generate-import-lib"))' -C metadata=9ad2a4f2678f35bd -C extra-filename=-9ad2a4f2678f35bd --out-dir /home/trent/src/tiktoken/target/release/deps -C linker=/home/trent/mambaforge/envs/py313t/bin/x86_64-conda-linux-gnu-cc -C strip=debuginfo -L dependency=/home/trent/src/tiktoken/target/release/deps --extern libc=/home/trent/src/tiktoken/target/release/deps/liblibc-6b0fbe5bd5ba9d30.rmeta --cap-lints allow --cfg Py_3_7 --cfg Py_3_8 --cfg Py_3_9 --cfg Py_3_10 --cfg Py_3_11 --cfg Py_3_12 --cfg Py_3_13 --cfg Py_GIL_DISABLED --cfg rustc_has_once_lock --cfg invalid_from_utf8_lint --cfg c_str_lit --cfg diagnostic_namespace --check-cfg 'cfg(Py_LIMITED_API)' --check-cfg 'cfg(Py_GIL_DISABLED)' --check-cfg 'cfg(PyPy)' --check-cfg 'cfg(GraalPy)' --check-cfg 'cfg(py_sys_config, values("Py_DEBUG", "Py_REF_DEBUG", "Py_TRACE_REFS", "COUNT_ALLOCS"))' --check-cfg 'cfg(invalid_from_utf8_lint)' --check-cfg 'cfg(pyo3_disable_reference_pool)' --check-cfg 'cfg(pyo3_leak_on_drop_without_reference_pool)' --check-cfg 'cfg(diagnostic_namespace)' --check-cfg 'cfg(c_str_lit)' --check-cfg 'cfg(rustc_has_once_lock)' --check-cfg 'cfg(Py_3_7)' --check-cfg 'cfg(Py_3_8)' --check-cfg 'cfg(Py_3_9)' --check-cfg 'cfg(Py_3_10)' --check-cfg 'cfg(Py_3_11)' --check-cfg 'cfg(Py_3_12)' --check-cfg 'cfg(Py_3_13)'`
     Running `rustc --crate-name pyo3_macros_backend --edition=2021 /home/trent/.cargo/registry/src/index.crates.io-6f17d22bba15001f/pyo3-macros-backend-0.23.3/src/lib.rs --error-format=json --json=diagnostic-rendered-ansi,artifacts,future-incompat --diagnostic-width=254 --crate-type lib --emit=dep-info,metadata,link -C embed-bitcode=no --warn=rust_2018_idioms '--warn=clippy::useless_transmute' '--warn=clippy::used_underscore_binding' --warn=unused_lifetimes '--warn=clippy::unnecessary_wraps' '--warn=clippy::todo' --warn=rust_2021_prelude_collisions '--warn=clippy::manual_ok_or' '--warn=clippy::manual_assert' '--warn=clippy::let_unit_value' --warn=invalid_doc_attributes '--warn=clippy::flat_map_option' '--warn=clippy::filter_map_next' '--warn=clippy::explicit_iter_loop' '--warn=clippy::explicit_into_iter_loop' --warn=elided_lifetimes_in_paths '--warn=clippy::dbg_macro' '--warn=clippy::checked_conversions' '--warn=rustdoc::broken_intra_doc_links' '--warn=rustdoc::bare_urls' -C debug-assertions=off --check-cfg 'cfg(docsrs)' --check-cfg 'cfg(feature, values("experimental-async"))' -C metadata=c7424188824c71f8 -C extra-filename=-c7424188824c71f8 --out-dir /home/trent/src/tiktoken/target/release/deps -C linker=/home/trent/mambaforge/envs/py313t/bin/x86_64-conda-linux-gnu-cc -C strip=debuginfo -L dependency=/home/trent/src/tiktoken/target/release/deps --extern heck=/home/trent/src/tiktoken/target/release/deps/libheck-5e22b1dffa7f4255.rmeta --extern proc_macro2=/home/trent/src/tiktoken/target/release/deps/libproc_macro2-4c69c42d9df03375.rmeta --extern pyo3_build_config=/home/trent/src/tiktoken/target/release/deps/libpyo3_build_config-b884267014f5753b.rmeta --extern quote=/home/trent/src/tiktoken/target/release/deps/libquote-169332b6fe3d21d6.rmeta --extern syn=/home/trent/src/tiktoken/target/release/deps/libsyn-29d4a0ddbd98f61f.rmeta --cap-lints allow --cfg rustc_has_once_lock --cfg invalid_from_utf8_lint --cfg c_str_lit --cfg diagnostic_namespace --check-cfg 'cfg(Py_LIMITED_API)' --check-cfg 'cfg(Py_GIL_DISABLED)' --check-cfg 'cfg(PyPy)' --check-cfg 'cfg(GraalPy)' --check-cfg 'cfg(py_sys_config, values("Py_DEBUG", "Py_REF_DEBUG", "Py_TRACE_REFS", "COUNT_ALLOCS"))' --check-cfg 'cfg(invalid_from_utf8_lint)' --check-cfg 'cfg(pyo3_disable_reference_pool)' --check-cfg 'cfg(pyo3_leak_on_drop_without_reference_pool)' --check-cfg 'cfg(diagnostic_namespace)' --check-cfg 'cfg(c_str_lit)' --check-cfg 'cfg(rustc_has_once_lock)' --check-cfg 'cfg(Py_3_7)' --check-cfg 'cfg(Py_3_8)' --check-cfg 'cfg(Py_3_9)' --check-cfg 'cfg(Py_3_10)' --check-cfg 'cfg(Py_3_11)' --check-cfg 'cfg(Py_3_12)' --check-cfg 'cfg(Py_3_13)'`
   Compiling fancy-regex v0.13.0
   Compiling regex v1.11.1
   Compiling bstr v1.11.3
     Running `rustc --crate-name regex --edition=2021 /home/trent/.cargo/registry/src/index.crates.io-6f17d22bba15001f/regex-1.11.1/src/lib.rs --error-format=json --json=diagnostic-rendered-ansi,artifacts,future-incompat --diagnostic-width=254 --crate-type lib --emit=dep-info,metadata,link -C opt-level=3 -C embed-bitcode=no --cfg 'feature="default"' --cfg 'feature="perf"' --cfg 'feature="perf-backtrack"' --cfg 'feature="perf-cache"' --cfg 'feature="perf-dfa"' --cfg 'feature="perf-inline"' --cfg 'feature="perf-literal"' --cfg 'feature="perf-onepass"' --cfg 'feature="std"' --cfg 'feature="unicode"' --cfg 'feature="unicode-age"' --cfg 'feature="unicode-bool"' --cfg 'feature="unicode-case"' --cfg 'feature="unicode-gencat"' --cfg 'feature="unicode-perl"' --cfg 'feature="unicode-script"' --cfg 'feature="unicode-segment"' --check-cfg 'cfg(docsrs)' --check-cfg 'cfg(feature, values("default", "logging", "pattern", "perf", "perf-backtrack", "perf-cache", "perf-dfa", "perf-dfa-full", "perf-inline", "perf-literal", "perf-onepass", "std", "unicode", "unicode-age", "unicode-bool", "unicode-case", "unicode-gencat", "unicode-perl", "unicode-script", "unicode-segment", "unstable", "use_std"))' -C metadata=9ab83d1dbb2872e1 -C extra-filename=-9ab83d1dbb2872e1 --out-dir /home/trent/src/tiktoken/target/release/deps -C linker=/home/trent/mambaforge/envs/py313t/bin/x86_64-conda-linux-gnu-cc -C strip=debuginfo -L dependency=/home/trent/src/tiktoken/target/release/deps --extern aho_corasick=/home/trent/src/tiktoken/target/release/deps/libaho_corasick-6668c9f838fb89b3.rmeta --extern memchr=/home/trent/src/tiktoken/target/release/deps/libmemchr-25fa9792dd9399b0.rmeta --extern regex_automata=/home/trent/src/tiktoken/target/release/deps/libregex_automata-01135b6ed6d4413e.rmeta --extern regex_syntax=/home/trent/src/tiktoken/target/release/deps/libregex_syntax-66f570e05dbe3825.rmeta --cap-lints allow`
     Running `rustc --crate-name fancy_regex --edition=2018 /home/trent/.cargo/registry/src/index.crates.io-6f17d22bba15001f/fancy-regex-0.13.0/src/lib.rs --error-format=json --json=diagnostic-rendered-ansi,artifacts,future-incompat --diagnostic-width=254 --crate-type lib --emit=dep-info,metadata,link -C opt-level=3 -C embed-bitcode=no --cfg 'feature="default"' --cfg 'feature="perf"' --cfg 'feature="std"' --cfg 'feature="unicode"' --check-cfg 'cfg(docsrs)' --check-cfg 'cfg(feature, values("default", "perf", "std", "track_caller", "unicode"))' -C metadata=2bd42caf041ad10c -C extra-filename=-2bd42caf041ad10c --out-dir /home/trent/src/tiktoken/target/release/deps -C linker=/home/trent/mambaforge/envs/py313t/bin/x86_64-conda-linux-gnu-cc -C strip=debuginfo -L dependency=/home/trent/src/tiktoken/target/release/deps --extern bit_set=/home/trent/src/tiktoken/target/release/deps/libbit_set-40d90f83eb5bab57.rmeta --extern regex_automata=/home/trent/src/tiktoken/target/release/deps/libregex_automata-01135b6ed6d4413e.rmeta --extern regex_syntax=/home/trent/src/tiktoken/target/release/deps/libregex_syntax-66f570e05dbe3825.rmeta --cap-lints allow`
     Running `rustc --crate-name bstr --edition=2021 /home/trent/.cargo/registry/src/index.crates.io-6f17d22bba15001f/bstr-1.11.3/src/lib.rs --error-format=json --json=diagnostic-rendered-ansi,artifacts,future-incompat --diagnostic-width=254 --crate-type lib --emit=dep-info,metadata,link -C opt-level=3 -C embed-bitcode=no --cfg 'feature="alloc"' --cfg 'feature="default"' --cfg 'feature="std"' --cfg 'feature="unicode"' --check-cfg 'cfg(docsrs)' --check-cfg 'cfg(feature, values("alloc", "default", "serde", "std", "unicode"))' -C metadata=cc96e78f4e9fd97f -C extra-filename=-cc96e78f4e9fd97f --out-dir /home/trent/src/tiktoken/target/release/deps -C linker=/home/trent/mambaforge/envs/py313t/bin/x86_64-conda-linux-gnu-cc -C strip=debuginfo -L dependency=/home/trent/src/tiktoken/target/release/deps --extern memchr=/home/trent/src/tiktoken/target/release/deps/libmemchr-25fa9792dd9399b0.rmeta --extern regex_automata=/home/trent/src/tiktoken/target/release/deps/libregex_automata-01135b6ed6d4413e.rmeta --cap-lints allow`
   Compiling pyo3-macros v0.23.3
     Running `rustc --crate-name pyo3_macros --edition=2021 /home/trent/.cargo/registry/src/index.crates.io-6f17d22bba15001f/pyo3-macros-0.23.3/src/lib.rs --error-format=json --json=diagnostic-rendered-ansi,artifacts,future-incompat --diagnostic-width=254 --crate-type proc-macro --emit=dep-info,link -C prefer-dynamic -C embed-bitcode=no --warn=rust_2018_idioms '--warn=clippy::useless_transmute' '--warn=clippy::used_underscore_binding' --warn=unused_lifetimes '--warn=clippy::unnecessary_wraps' '--warn=clippy::todo' --warn=rust_2021_prelude_collisions '--warn=clippy::manual_ok_or' '--warn=clippy::manual_assert' '--warn=clippy::let_unit_value' --warn=invalid_doc_attributes '--warn=clippy::flat_map_option' '--warn=clippy::filter_map_next' '--warn=clippy::explicit_iter_loop' '--warn=clippy::explicit_into_iter_loop' --warn=elided_lifetimes_in_paths '--warn=clippy::dbg_macro' '--warn=clippy::checked_conversions' '--warn=rustdoc::broken_intra_doc_links' '--warn=rustdoc::bare_urls' -C debug-assertions=off --check-cfg 'cfg(docsrs)' --check-cfg 'cfg(feature, values("experimental-async", "multiple-pymethods"))' -C metadata=97a13ca0f34dda72 -C extra-filename=-97a13ca0f34dda72 --out-dir /home/trent/src/tiktoken/target/release/deps -C linker=/home/trent/mambaforge/envs/py313t/bin/x86_64-conda-linux-gnu-cc -C strip=debuginfo -L dependency=/home/trent/src/tiktoken/target/release/deps --extern proc_macro2=/home/trent/src/tiktoken/target/release/deps/libproc_macro2-4c69c42d9df03375.rlib --extern pyo3_macros_backend=/home/trent/src/tiktoken/target/release/deps/libpyo3_macros_backend-c7424188824c71f8.rlib --extern quote=/home/trent/src/tiktoken/target/release/deps/libquote-169332b6fe3d21d6.rlib --extern syn=/home/trent/src/tiktoken/target/release/deps/libsyn-29d4a0ddbd98f61f.rlib --extern proc_macro --cap-lints allow`
     Running `rustc --crate-name pyo3 --edition=2021 /home/trent/.cargo/registry/src/index.crates.io-6f17d22bba15001f/pyo3-0.23.3/src/lib.rs --error-format=json --json=diagnostic-rendered-ansi,artifacts,future-incompat --diagnostic-width=254 --crate-type lib --emit=dep-info,metadata,link -C opt-level=3 -C embed-bitcode=no --warn=rust_2018_idioms '--warn=clippy::useless_transmute' '--warn=clippy::used_underscore_binding' --warn=unused_lifetimes '--warn=clippy::unnecessary_wraps' '--warn=clippy::todo' --warn=rust_2021_prelude_collisions '--warn=clippy::manual_ok_or' '--warn=clippy::manual_assert' '--warn=clippy::let_unit_value' --warn=invalid_doc_attributes '--warn=clippy::flat_map_option' '--warn=clippy::filter_map_next' '--warn=clippy::explicit_iter_loop' '--warn=clippy::explicit_into_iter_loop' --warn=elided_lifetimes_in_paths '--warn=clippy::dbg_macro' '--warn=clippy::checked_conversions' '--warn=rustdoc::broken_intra_doc_links' '--warn=rustdoc::bare_urls' --cfg 'feature="extension-module"' --cfg 'feature="indoc"' --cfg 'feature="macros"' --cfg 'feature="pyo3-macros"' --cfg 'feature="unindent"' --check-cfg 'cfg(docsrs)' --check-cfg 'cfg(feature, values("abi3", "abi3-py310", "abi3-py311", "abi3-py312", "abi3-py37", "abi3-py38", "abi3-py39", "anyhow", "auto-initialize", "chrono", "chrono-tz", "default", "either", "experimental-async", "experimental-inspect", "extension-module", "eyre", "full", "generate-import-lib", "hashbrown", "indexmap", "indoc", "inventory", "macros", "multiple-pymethods", "nightly", "num-bigint", "num-complex", "num-rational", "py-clone", "pyo3-macros", "rust_decimal", "serde", "smallvec", "unindent"))' -C metadata=99d0b007138eadf4 -C extra-filename=-99d0b007138eadf4 --out-dir /home/trent/src/tiktoken/target/release/deps -C linker=/home/trent/mambaforge/envs/py313t/bin/x86_64-conda-linux-gnu-cc -C strip=debuginfo -L dependency=/home/trent/src/tiktoken/target/release/deps --extern cfg_if=/home/trent/src/tiktoken/target/release/deps/libcfg_if-e4ded2c19830fbdd.rmeta --extern indoc=/home/trent/src/tiktoken/target/release/deps/libindoc-b4e94d9ecbd21a39.so --extern libc=/home/trent/src/tiktoken/target/release/deps/liblibc-6b0fbe5bd5ba9d30.rmeta --extern memoffset=/home/trent/src/tiktoken/target/release/deps/libmemoffset-65fe1a8a113b3005.rmeta --extern once_cell=/home/trent/src/tiktoken/target/release/deps/libonce_cell-05003007543cfa87.rmeta --extern pyo3_ffi=/home/trent/src/tiktoken/target/release/deps/libpyo3_ffi-9ad2a4f2678f35bd.rmeta --extern pyo3_macros=/home/trent/src/tiktoken/target/release/deps/libpyo3_macros-97a13ca0f34dda72.so --extern unindent=/home/trent/src/tiktoken/target/release/deps/libunindent-d53e2a0385a47a80.rmeta --cap-lints allow --cfg Py_3_7 --cfg Py_3_8 --cfg Py_3_9 --cfg Py_3_10 --cfg Py_3_11 --cfg Py_3_12 --cfg Py_3_13 --cfg Py_GIL_DISABLED --cfg rustc_has_once_lock --cfg invalid_from_utf8_lint --cfg c_str_lit --cfg diagnostic_namespace --check-cfg 'cfg(Py_LIMITED_API)' --check-cfg 'cfg(Py_GIL_DISABLED)' --check-cfg 'cfg(PyPy)' --check-cfg 'cfg(GraalPy)' --check-cfg 'cfg(py_sys_config, values("Py_DEBUG", "Py_REF_DEBUG", "Py_TRACE_REFS", "COUNT_ALLOCS"))' --check-cfg 'cfg(invalid_from_utf8_lint)' --check-cfg 'cfg(pyo3_disable_reference_pool)' --check-cfg 'cfg(pyo3_leak_on_drop_without_reference_pool)' --check-cfg 'cfg(diagnostic_namespace)' --check-cfg 'cfg(c_str_lit)' --check-cfg 'cfg(rustc_has_once_lock)' --check-cfg 'cfg(Py_3_7)' --check-cfg 'cfg(Py_3_8)' --check-cfg 'cfg(Py_3_9)' --check-cfg 'cfg(Py_3_10)' --check-cfg 'cfg(Py_3_11)' --check-cfg 'cfg(Py_3_12)' --check-cfg 'cfg(Py_3_13)'`
       Dirty tiktoken v0.8.0 (/home/trent/src/tiktoken): the toolchain changed
   Compiling tiktoken v0.8.0 (/home/trent/src/tiktoken)
     Running `rustc --crate-name _tiktoken --edition=2021 src/lib.rs --error-format=json --json=diagnostic-rendered-ansi,artifacts,future-incompat --diagnostic-width=254 --crate-type cdylib --emit=dep-info,link -C opt-level=3 -C embed-bitcode=no --check-cfg 'cfg(docsrs)' --check-cfg 'cfg(feature, values())' -C metadata=2d15c1d1b98ec97b --out-dir /home/trent/src/tiktoken/target/release/deps -C linker=/home/trent/mambaforge/envs/py313t/bin/x86_64-conda-linux-gnu-cc -C strip=debuginfo -L dependency=/home/trent/src/tiktoken/target/release/deps --extern bstr=/home/trent/src/tiktoken/target/release/deps/libbstr-cc96e78f4e9fd97f.rlib --extern fancy_regex=/home/trent/src/tiktoken/target/release/deps/libfancy_regex-2bd42caf041ad10c.rlib --extern pyo3=/home/trent/src/tiktoken/target/release/deps/libpyo3-99d0b007138eadf4.rlib --extern regex=/home/trent/src/tiktoken/target/release/deps/libregex-9ab83d1dbb2872e1.rlib --extern rustc_hash=/home/trent/src/tiktoken/target/release/deps/librustc_hash-b006f4d81e95dfaf.rlib`
warning: use of deprecated method `pyo3::IntoPy::into_py`: `IntoPy` is going to be replaced by `IntoPyObject`. See the migration guide (https://pyo3.rs/v0.23.0/migration) for more information.
   --> src/lib.rs:508:16
    |
508 |         buffer.into_py(py)
    |                ^^^^^^^
    |
    = note: `#[warn(deprecated)]` on by default

warning: use of deprecated associated function `pyo3::types::PyList::new_bound`: renamed to `PyList::new`
   --> src/lib.rs:555:38
    |
555 |         let py_completions = PyList::new_bound(
    |                                      ^^^^^^^^^

warning: use of deprecated associated function `pyo3::types::PyList::new_bound`: renamed to `PyList::new`
   --> src/lib.rs:559:36
    |
559 |                 .map(|seq| PyList::new_bound(py, &seq[..])),
    |                                    ^^^^^^^^^

warning: use of deprecated method `pyo3::IntoPy::into_py`: `IntoPy` is going to be replaced by `IntoPyObject`. See the migration guide (https://pyo3.rs/v0.23.0/migration) for more information.
   --> src/lib.rs:561:34
    |
561 |         (tokens, py_completions).into_py(py)
    |                                  ^^^^^^^

warning: use of deprecated associated function `pyo3::types::PyBytes::new_bound`: renamed to `PyBytes::new`
   --> src/lib.rs:589:38
    |
589 |             Ok(bytes) => Ok(PyBytes::new_bound(py, &bytes).into()),
    |                                      ^^^^^^^^^

warning: use of deprecated associated function `pyo3::types::PyBytes::new_bound`: renamed to `PyBytes::new`
   --> src/lib.rs:596:32
    |
596 |             return Ok(PyBytes::new_bound(py, bytes).into());
    |                                ^^^^^^^^^

warning: use of deprecated associated function `pyo3::types::PyBytes::new_bound`: renamed to `PyBytes::new`
   --> src/lib.rs:599:32
    |
599 |             return Ok(PyBytes::new_bound(py, bytes).into());
    |                                ^^^^^^^^^

warning: use of deprecated associated function `pyo3::types::PyBytes::new_bound`: renamed to `PyBytes::new`
   --> src/lib.rs:611:31
    |
611 |             .map(|x| PyBytes::new_bound(py, x).into())
    |                               ^^^^^^^^^

warning: `tiktoken` (lib) generated 8 warnings
    Finished `release` profile [optimized] target(s) in 13.17s
Copying rust artifact from target/release/lib_tiktoken.so to build/lib.linux-x86_64-cpython-313t/tiktoken/_tiktoken.cpython-313t-x86_64-linux-gnu.so
(py313t) {pytorch} [12.6] [trent@dgx/ttypts/3(~s/tiktoken)%] python setup.py install
running install
/home/trent/mambaforge/envs/py313t/lib/python3.13t/site-packages/setuptools/_distutils/cmd.py:79: SetuptoolsDeprecationWarning: setup.py install is deprecated.
!!

        ********************************************************************************
        Please avoid running ``setup.py`` directly.
        Instead, use pypa/build, pypa/installer or other
        standards-based tools.

        See https://blog.ganssle.io/articles/2021/10/setup-py-deprecated.html for details.
        ********************************************************************************

!!
  self.initialize_options()
running build
running build_py
copying tiktoken/_educational.py -> build/lib.linux-x86_64-cpython-313t/tiktoken
copying tiktoken/__init__.py -> build/lib.linux-x86_64-cpython-313t/tiktoken
copying tiktoken/registry.py -> build/lib.linux-x86_64-cpython-313t/tiktoken
copying tiktoken/model.py -> build/lib.linux-x86_64-cpython-313t/tiktoken
copying tiktoken/load.py -> build/lib.linux-x86_64-cpython-313t/tiktoken
copying tiktoken/core.py -> build/lib.linux-x86_64-cpython-313t/tiktoken
copying tiktoken_ext/openai_public.py -> build/lib.linux-x86_64-cpython-313t/tiktoken_ext
running egg_info
writing tiktoken.egg-info/PKG-INFO
writing dependency_links to tiktoken.egg-info/dependency_links.txt
writing requirements to tiktoken.egg-info/requires.txt
writing top-level names to tiktoken.egg-info/top_level.txt
reading manifest file 'tiktoken.egg-info/SOURCES.txt'
reading manifest template 'MANIFEST.in'
warning: no files found matching 'Makefile'
adding license file 'LICENSE'
writing manifest file 'tiktoken.egg-info/SOURCES.txt'
copying tiktoken/py.typed -> build/lib.linux-x86_64-cpython-313t/tiktoken
running build_ext
running build_rust
cargo rustc --lib --message-format=json-render-diagnostics --manifest-path Cargo.toml --release -v --features pyo3/extension-module --crate-type cdylib --
       Fresh target-lexicon v0.12.16
       Fresh unicode-ident v1.0.14
       Fresh memchr v2.7.4
       Fresh regex-syntax v0.8.5
       Fresh autocfg v1.4.0
       Fresh aho-corasick v1.1.3
       Fresh heck v0.5.0
       Fresh bit-vec v0.6.3
       Fresh indoc v2.0.5
       Fresh cfg-if v1.0.0
       Fresh once_cell v1.20.2
       Fresh unindent v0.2.3
       Fresh rustc-hash v1.1.0
       Fresh proc-macro2 v1.0.92
       Fresh regex-automata v0.4.9
       Fresh libc v0.2.169
       Fresh bit-set v0.5.3
       Fresh pyo3-build-config v0.23.3
       Fresh quote v1.0.38
       Fresh memoffset v0.9.1
       Fresh bstr v1.11.3
       Fresh fancy-regex v0.13.0
       Fresh regex v1.11.1
       Fresh syn v2.0.95
       Fresh pyo3-macros-backend v0.23.3
       Fresh pyo3-ffi v0.23.3
       Fresh pyo3-macros v0.23.3
       Fresh pyo3 v0.23.3
       Fresh tiktoken v0.8.0 (/home/trent/src/tiktoken)
warning: use of deprecated method `pyo3::IntoPy::into_py`: `IntoPy` is going to be replaced by `IntoPyObject`. See the migration guide (https://pyo3.rs/v0.23.0/migration) for more information.
   --> src/lib.rs:508:16
    |
508 |         buffer.into_py(py)
    |                ^^^^^^^
    |
    = note: `#[warn(deprecated)]` on by default

warning: use of deprecated associated function `pyo3::types::PyList::new_bound`: renamed to `PyList::new`
   --> src/lib.rs:555:38
    |
555 |         let py_completions = PyList::new_bound(
    |                                      ^^^^^^^^^

warning: use of deprecated associated function `pyo3::types::PyList::new_bound`: renamed to `PyList::new`
   --> src/lib.rs:559:36
    |
559 |                 .map(|seq| PyList::new_bound(py, &seq[..])),
    |                                    ^^^^^^^^^

warning: use of deprecated method `pyo3::IntoPy::into_py`: `IntoPy` is going to be replaced by `IntoPyObject`. See the migration guide (https://pyo3.rs/v0.23.0/migration) for more information.
   --> src/lib.rs:561:34
    |
561 |         (tokens, py_completions).into_py(py)
    |                                  ^^^^^^^

warning: use of deprecated associated function `pyo3::types::PyBytes::new_bound`: renamed to `PyBytes::new`
   --> src/lib.rs:589:38
    |
589 |             Ok(bytes) => Ok(PyBytes::new_bound(py, &bytes).into()),
    |                                      ^^^^^^^^^

warning: use of deprecated associated function `pyo3::types::PyBytes::new_bound`: renamed to `PyBytes::new`
   --> src/lib.rs:596:32
    |
596 |             return Ok(PyBytes::new_bound(py, bytes).into());
    |                                ^^^^^^^^^

warning: use of deprecated associated function `pyo3::types::PyBytes::new_bound`: renamed to `PyBytes::new`
   --> src/lib.rs:599:32
    |
599 |             return Ok(PyBytes::new_bound(py, bytes).into());
    |                                ^^^^^^^^^

warning: use of deprecated associated function `pyo3::types::PyBytes::new_bound`: renamed to `PyBytes::new`
   --> src/lib.rs:611:31
    |
611 |             .map(|x| PyBytes::new_bound(py, x).into())
    |                               ^^^^^^^^^

warning: `tiktoken` (lib) generated 8 warnings
    Finished `release` profile [optimized] target(s) in 0.03s
Copying rust artifact from target/release/lib_tiktoken.so to build/lib.linux-x86_64-cpython-313t/tiktoken/_tiktoken.cpython-313t-x86_64-linux-gnu.so
running install_lib
creating /home/trent/mambaforge/envs/py313t/lib/python3.13t/site-packages/tiktoken_ext
copying build/lib.linux-x86_64-cpython-313t/tiktoken_ext/openai_public.py -> /home/trent/mambaforge/envs/py313t/lib/python3.13t/site-packages/tiktoken_ext
creating /home/trent/mambaforge/envs/py313t/lib/python3.13t/site-packages/tiktoken
copying build/lib.linux-x86_64-cpython-313t/tiktoken/_educational.py -> /home/trent/mambaforge/envs/py313t/lib/python3.13t/site-packages/tiktoken
copying build/lib.linux-x86_64-cpython-313t/tiktoken/__init__.py -> /home/trent/mambaforge/envs/py313t/lib/python3.13t/site-packages/tiktoken
copying build/lib.linux-x86_64-cpython-313t/tiktoken/py.typed -> /home/trent/mambaforge/envs/py313t/lib/python3.13t/site-packages/tiktoken
copying build/lib.linux-x86_64-cpython-313t/tiktoken/registry.py -> /home/trent/mambaforge/envs/py313t/lib/python3.13t/site-packages/tiktoken
copying build/lib.linux-x86_64-cpython-313t/tiktoken/model.py -> /home/trent/mambaforge/envs/py313t/lib/python3.13t/site-packages/tiktoken
copying build/lib.linux-x86_64-cpython-313t/tiktoken/load.py -> /home/trent/mambaforge/envs/py313t/lib/python3.13t/site-packages/tiktoken
copying build/lib.linux-x86_64-cpython-313t/tiktoken/_tiktoken.cpython-313t-x86_64-linux-gnu.so -> /home/trent/mambaforge/envs/py313t/lib/python3.13t/site-packages/tiktoken
copying build/lib.linux-x86_64-cpython-313t/tiktoken/core.py -> /home/trent/mambaforge/envs/py313t/lib/python3.13t/site-packages/tiktoken
byte-compiling /home/trent/mambaforge/envs/py313t/lib/python3.13t/site-packages/tiktoken_ext/openai_public.py to openai_public.cpython-313.pyc
byte-compiling /home/trent/mambaforge/envs/py313t/lib/python3.13t/site-packages/tiktoken/_educational.py to _educational.cpython-313.pyc
byte-compiling /home/trent/mambaforge/envs/py313t/lib/python3.13t/site-packages/tiktoken/__init__.py to __init__.cpython-313.pyc
byte-compiling /home/trent/mambaforge/envs/py313t/lib/python3.13t/site-packages/tiktoken/registry.py to registry.cpython-313.pyc
byte-compiling /home/trent/mambaforge/envs/py313t/lib/python3.13t/site-packages/tiktoken/model.py to model.cpython-313.pyc
byte-compiling /home/trent/mambaforge/envs/py313t/lib/python3.13t/site-packages/tiktoken/load.py to load.cpython-313.pyc
byte-compiling /home/trent/mambaforge/envs/py313t/lib/python3.13t/site-packages/tiktoken/core.py to core.cpython-313.pyc
running install_egg_info
Copying tiktoken.egg-info to /home/trent/mambaforge/envs/py313t/lib/python3.13t/site-packages/tiktoken-0.8.0-py3.13.egg-info
running install_scripts
```



:::

After this, you should be able to import the `tiktoken` module in Python:

```bash
% cd ..
% python -Xgil=0
Python 3.13.1 experimental free-threading build | packaged by conda-forge | (main, Jan 13 2025, 09:59:40) [GCC 13.3.0] on linux
Type "help", "copyright", "credits" or "license" for more information.
>>> import tiktoken
>>>
```

#### Torch

Install PyTorch 2.6 via `pip` with the conda `py313t` environment active:

```bash
pip install torch==2.6 --index-url https://download.pytorch.org/whl/cu126
```

If you have trouble installing PyTorch, consult their [Getting Started](
https://pytorch.org/get-started/locally/) guide.

You can verify torch installed correctly as follows:

```bash
% python -Xgil=0
Python 3.13.1 experimental free-threading build | packaged by conda-forge | (main, Jan 13 2025, 09:59:40) [GCC 13.3.0] on linux
Type "help", "copyright", "credits" or "license" for more information.
>>> import torch
>>> torch.cuda.is_available()
True
```

#### IPython Kernel

Installing IPython Kernel allows us to use our free-threaded Python
installation via the Jupyter Lab instance we install in the `py313`
environment.

```bash
conda activate py313t
pip install ipykernel
```

Once `ipykernel` is installed, run the following:

```bash
% python3.13t -m ipykernel --install --name py313t --user
Installed kernelspec py313t in /home/trent/.local/share/jupyter/kernels/py313t
```

This will install a kernel configuration in
`~/.local/jupyter/share/kernels/py313t`, which we then need to tweak by adding
the `-Xgil=0` startup flag:

```bash
% cd ~/.local/jupyter/share/kernels/py313t
% cp kernel.json kernel.json.orig
% vi kernel.json
# Edit kernel.json to make it look like the diff below.
```

```diff
--- kernel.json.orig    2025-02-04 15:02:21.814112004 -0800
+++ kernel.json 2025-02-04 15:02:36.553806199 -0800
@@ -1,6 +1,7 @@
 {
  "argv": [
   "/home/trent/mambaforge/envs/py313t/bin/python3.13t",
+  "-Xgil=0",
   "-Xfrozen_modules=off",
   "-m",
   "ipykernel_launcher",
@@ -12,4 +13,4 @@
  "metadata": {
   "debugger": true
  }
```

#### Datrie and Cython

[datrie](https://github.com/pytries/datrie) is a Python library that provides
a *trie* (or *digital search tree*) data structure by way of the [libdatrie](
https://linux.thai.net/~thep/datrie/datrie.html) C library.  The Python
`datrie` library isn't strictly necessary to run `parallelopedia.gpt2`, but
other components rely on it, so it's handy to get installed now, if possible.

It relies upon Cython, and thus, for now, you need to install a free-threaded
compatible version of Cython first, as follows:

```bash
conda activate py313t
pip install git+https://github.com/cython/cython
```
Then, clone the `datrie` repo and install as follows:

```bash
conda activate py313t
git clone https://github.com/pytries/datrie --recursive
cd datrie
python setup.py build
python setup.py install
```

If everything goes well, you should see something like this when you launch
Python and import `datrie`:

```bash
% python
Python 3.13.1 experimental free-threading build | packaged by conda-forge | (main, Jan 13 2025, 09:59:40) [GCC 13.3.0] on linux
Type "help", "copyright", "credits" or "license" for more information.
>>> import datrie
>>>
```

### Normal 3.13 Env (py313)

The second `py313` environment is almost identical to `py313t`, except it is
not a `python-freethreading` installation, and, additionally, we install
Jupyter Lab.  We can install `tiktoken` directly via `pip`, so we don't need
the supporting Rust cruft.  Likewise for `datrie`, we don't need to first
install Cython and then build `datrie` from git.

```bash
conda create -n py313 python=3.13 \
    nodejs pip tqdm flake8 jupyterlab \
        -c conda-forge
conda activate py313
pip install numpy datrie tiktoken
pip install torch==2.6 --index-url https://download.pytorch.org/whl/cu126
```

## Parallelopedia

All of the code in this article is available in the [Parallelopedia](
https://github.com/tpn/parallelopedia) repository on Github.  The code we'll
be focusing on in this post lives in the [parallelopedia.gpt2](
https://github.com/tpn/parallelopedia/blob/main/src/parallelopedia/gpt2.py)
module.

Clone the repository as follows:

```bash
git clone https://github.com/tpn/parallelopedia
```

The code and command examples in this post will assume you've added the
`src` directory to your `PYTHONPATH`, and the `bin` directory to your `PATH`:

```bash
cd parallelopedia
export PYTHONPATH=$(pwd)/src:$PYTHONPATH
export PATH=$(pwd)/bin:$PATH

```

You can perform a quick sanity check that things are working as follows:

```bash
% python -Xgil=0 -m parallelopedia.http.server --help
usage: server.py [-h] [--ip IP] [--port PORT] [--debug] [--log-level {DEBUG,INFO,WARNING,ERROR,CRITICAL}] [--threads THREADS] [--protocol-class PROTOCOL_CLASS] [--app-classes APP_CLASSES [APP_CLASSES ...]] [--listen-backlog LISTEN_BACKLOG]

Run the HTTP server.

options:
  -h, --help            show this help message and exit
  --ip IP               IP address to bind the server to.
  --port PORT           Port number to bind the server to.
  --debug               Enable debug mode for asyncio.
  --log-level {DEBUG,INFO,WARNING,ERROR,CRITICAL}
                        Set the logging level.
  --threads THREADS     Number of threads to use.
  --protocol-class PROTOCOL_CLASS
                        The protocol class to use for the server.
  --app-classes APP_CLASSES [APP_CLASSES ...]
                        Space-separated list of HTTP application classes.
  --listen-backlog LISTEN_BACKLOG
                        The listen backlog for the server.

```

## Parallelopedia Web Interface

The React Bootstrap web interface lives in the [Parallelopedia-UI](
https://github.com/tpn/parallelopedia-ui) repository.

::: {.callout-note}

Full disclaimer: I'm not a web developer.  I don't know JavaScript, React,
or Bootstrap, or anything else in the modern web stack.  So, like any
modern developer in 2025, when faced with needing to whip up something in an
area I am not proficient, I farm it all out to AI---either ChatGPT, local
LLMs via [LM Studio](https://lmstudio.ai/), or, more recently, [Aider](
https://aider.chat).

TL;DR: the web interface code probably sucks.

:::

Clone the repository as follows:

```bash
git clone https://github.com/tpn/parallelopedia-ui
```

Both the `py313t` and `py313` environments included `nodejs`, so with either
of them active, you should be able to change into that directory and run
`npm run start` to start up the web interface:

```bash
conda activate py313
cd parallelopedia-ui
npm run start
```

That should launch a browser automatically to `http://localhost:3000/`, which
should have a `GPT2` tab that, when selected, looks something like this:

::: {.lightbox .theme-light}
![Parallelopedia UI Example](images/parallelopedia-ui-gpt2-light.png)
:::

::: {.lightbox .theme-dark}
![Parallelopedia UI Example](images/parallelopedia-ui-gpt2-dark.png)
:::

# Learning about LLMs

I started at [OpenTeams](https://openteams.com) around the middle of November,
2024 (last year), and joined the PyTorch team.  At the time I honestly knew
nothing about PyTorch.  Nor deep neural networks.  Nor LLMs---other than
knowing I thoroughly enjoyed using them, having been an avid ChatGPT user
for a while now.

Thanks to [Andrej Karpathy](https://karpathy.ai/)'s phenomenal YouTube series
on deep neural networks and LLMs titled [Neural Networks: From Zero to Hero](
https://www.youtube.com/playlist?list=PLAqhIrjkxbuWI23v9cThsA9GvCAUhRvKZ),
featuring 19 hours, 21 minutes and two seconds of content across ten videos,
over the course of about 3-4 weeks, I went from zero to... *hero* is too
strong of a word---I'd say I at least made it to *not-completely-clueless*
level with regards to my understandings of how LLMs work.  I was also soaking
up as much PyTorch exposure as I could throughout this period, which is made
easier given Andrej leverages PyTorch in later videos.

The content is simply fantastic.  Andrej's is a great teacher, and he does a
wonderful job of laying the foundation of modern neural networks, particularly
with regard to the *bread-and-butter* concepts like auto-differentiation and
back-propagation that fuel today's LLM AI models.

None of the work presented in this post would have been possible had I not
invested the time in Andrej's series.  If you're reading this Andrej, thanks,
and keep up the brilliant work!

## Training GPT-2 (124M) Locally

Equipped with my new knowledge about LLMs, PyTorch, and, thanks to Andrej's
final video in the series titled [Let's reproduce GPT-2 (124M)](
https://www.youtube.com/watch?v=l8pRSuU81PU&list=PLAqhIrjkxbuWI23v9cThsA9GvCAUhRvKZ&index=10&t=1286s&pp=iAQB)
and the accompanying [build-nanogpt](
https://github.com/karpathy/build-nanogpt) Github repo, I was able to train
a local GPT-2 model via PyTorch, from scratch, using the [edu_fineweb10B](
https://huggingface.co/rhysjones/gpt2-124M-edu-fineweb-10B) dataset.

I only had to make one change in order to run locally:
[Use 8 for micro batch size](https://github.com/tpn/build-nanogpt/commit/0069c4a35d1a362c1fd50f9f5bce00a170e15904).
With that change in place, I was able to train GPT-2 from scratch as follows:

```bash
conda activate py313
git clone gh:tpn/build-nanogpt
cd build-nanogpt
# Download the fineweb dataset.
python fineweb.py
# Train!
torchrun --standalone --nproc_per_node=4 train_gpt2.py
```

This was run on an NVIDIA DGX workstation from 2017, which has an Intel
Xeon(R) CPU E5-2698 v4 @ 2.20GHz (20 cores, 40 threads), and four Tesla
V100-DGXS-32GB GPUs.

Training in parallel across all four GPUs yielded around 36,000 tokens/sec,
with an average time of about 14.5 seconds per loop iteration.  Training
took about 3 days and 5 hours for 19,072 steps.  All four GPUs were pegged
close to their 300W power limit for those three days.

::: {.callout-note}

Amusingly, well after the fact, I decided to see what kind of training
performance I'd get on my Windows 11 gaming box, which has an AMD Ryzen 9
7950X3D (16 cores, 32 threads) and NVIDIA RTX 4090.  Training via
`python train_gpt2.py` (`torchrun` wasn't needed as I wasn't using multiple
GPUs) yielded about 45,000 tokens/sec, which is a nice bump, but what was most
impressive was the reduction to the loop iteration duration, which averaged
out to about 180ms.

So, I could have completed the same training process in about an hour or so,
at a vastly reduced impact on my electricity bill that month :-)

:::

Once training completes, a `log/model_19072.pt` file is produced, which is
the checkpoint of the model at that final step, obtained via a call to
`torch.save()`.  The model has 124M parameters---which is tiny by modern
standards---and is just under 500MB on disk.

You can download that very model I trained via the HuggingFace dataset I set
up here: [model_19072.pt](
https://huggingface.co/datasets/trentnelson/parallelopedia-data-gpt2/blob/main/model_19072.pt).
Once downloaded, place the file in `~/src/parallelopedia/data` (assuming you
cloned the repo to `~/src/parallelopedia`; change as necessary), and that will
be all that's required for the next steps.

# Parallel PyTorch Inference with Free-Threaded Python

Now we finally get to the fun part.  The primary objective I intended to
tackle with this work was to investigate whether or not PyTorch model
inference could be done simultaneously, by multiple threads, in a single
free-threaded Python process, ideally with just a single GPU at minimum.

I didn't focus on exploring free-threading and *training* PyTorch models.
Mainly because it's a lot more complex, plus there's already a huge body of
existing work in PyTorch that handles distributed training across multiple
GPUs via `multiprocessing`.  On the flip side, having multiple threads do
simultaneous parallel inference on a single model in a single Python process
is novel territory: it hasn't been possible prior to now, because of the
Python GIL.

::: {.hidden}
Additionally, parallel multi-threaded inference is particularly interesting
because you should arguably be able to have an HTTP server servicing *"chat"*
requests (i.e. like an OpenAI compatible REST API) in parallel with only one
GPU.  (Although no reason multiple GPUs can't also be leveraged, as we'll
find out.)
:::

## A Simple PyTorch GPT-2 Implementation

Let's introduce the first version of the Python code we're going to use.
Again, all of this has been made possible thanks to Andrej Karpathy's work
with his [YouTube series](
https://www.youtube.com/watch?v=l8pRSuU81PU&list=PLAqhIrjkxbuWI23v9cThsA9GvCAUhRvKZ&index=10&t=1286s&pp=iAQB)
and [build-nanogpt](
https://github.com/karpathy/build-nanogpt) repo, so any and all code you see
in this post can typically be traced back to something equivalent that appears
in [train_gpt2.py](
https://github.com/karpathy/build-nanogpt/blob/master/train_gpt2.py).  None
of this code would have made any sense to me a month or two ago---but I can
promise you that if you devote sufficient time to watching and understanding
the entire series, you'll grok every single line!

You can follow along in a Jupyter Lab notebook if you activate the `py313`
environment and launch `jupyter lab`.  If you correctly registered your
`py313t` kernel per the instructions earlier, you should see an option when
creating a new notebook to use the `py313t` Python kernel, which will be the
free-threaded version.

The code below roughly corresponds to my first version of the code in the
commit [3ed4fe6: Add gpt2.py](
https://github.com/tpn/parallelopedia/blob/3ed4fe60a767a12b31fca183fed00fef43c65827/src/parallelopedia/gpt2.py),
with some formatting and style tweaks to ensure the code is viewable on mobile
devices without requiring horizontal scrolling.

There are a number of deficiencies in this code, which we'll address in
subsequent versions.  For now, it's a good starting point to get a feel for
how we can use PyTorch to load a GPT-2 model checkpoint, tokenize some input
text, and generate some output text.


```{.python .numberLines}
# ===================================================================
# Imports
# ===================================================================
import dataclasses
import logging

import tiktoken
import torch
import torch.nn as nn
from torch.nn import functional as F

# ===================================================================
# Globals
# ===================================================================

# The following assumes you've created a notebook in the
# root of the parallelopedia repo, and you've downloaded
# the model_19072.pt file from HuggingFace per earlier
# instructions and placed it in the 'data' directory.
MODEL_CHECKPOINT = 'data/model_19072.pt'

LOG_LEVEL = 'DEBUG'

# ===================================================================
# Logging
# ===================================================================
logging.basicConfig(
    level=getattr(logging, LOG_LEVEL),
    format='%(asctime)s - %(levelname)s - %(message)s',
)

# ===================================================================
# Setup
# ===================================================================

# Use bfloat16 for matmul precision where possible.
torch.set_float32_matmul_precision('high')

# ===================================================================
# GPT2 PyTorch Model Components
# ===================================================================

# Now define the classes making up our GPT2 implementation.
# These map directly to the components introduced by the
# now-seminal 2017 "Attention Is All You Need" paper.

class CausalSelfAttention(nn.Module):
    """
    Causal self-attention for the GPT2 model.
    """

    def __init__(self, config):
        super().__init__()
        assert config.n_embd % config.n_head == 0
        # Key, query, value projections for all heads, but in a batch.
        self.c_attn = nn.Linear(config.n_embd, 3 * config.n_embd)

        # Output projection.
        self.c_proj = nn.Linear(config.n_embd, config.n_embd)
        self.c_proj.NANOGPT_SCALE_INIT = 1

        # Regularization.
        self.n_head = config.n_head
        self.n_embd = config.n_embd

    def forward(self, x):
        # Batch size, sequence length, embedding dimensionality.
        B, T, C = (x.size())

        # Calculate query, key, values for all heads in
        # batch and move head forward to be the batch dim.
        #
        # N.B. nh is "number of heads", hs is "head size",
        #      and C (number of channels) is nh * hs.
        #      E.g. in GPT-2 (124M), n_head=12, hs=64, so
        #      nh*hs=C=768 channels in the Transformer.
        qkv = self.c_attn(x)
        q, k, v = qkv.split(self.n_embd, dim=2)

        head_dim = C // self.n_head

        # (B, nh, T, hs)
        k = k.view(B, T, self.n_head, head_dim).transpose(1, 2)

        # (B, nh, T, hs)
        q = q.view(B, T, self.n_head, head_dim).transpose(1, 2)

        # (B, nh, T, hs)
        v = v.view(B, T, self.n_head, head_dim).transpose(1, 2)

        # Flash attention.
        y = F.scaled_dot_product_attention(q, k, v, is_causal=True)

        # Re-assemble all head outputs side by side.
        y = (y.transpose(1, 2).contiguous().view(B, T, C))

        # Output projection.
        y = self.c_proj(y)
        return y


class MLP(nn.Module):
    """
    Multi-layer perceptron for the GPT2 model.
    """

    def __init__(self, config):
        super().__init__()
        self.c_fc = nn.Linear(config.n_embd, 4 * config.n_embd)
        self.gelu = nn.GELU(approximate='tanh')
        self.c_proj = nn.Linear(4 * config.n_embd, config.n_embd)
        self.c_proj.NANOGPT_SCALE_INIT = 1

    def forward(self, x):
        x = self.c_fc(x)
        x = self.gelu(x)
        x = self.c_proj(x)
        return x


class Block(nn.Module):
    """
    Transformer block for the GPT2 model.
    """

    def __init__(self, config):
        super().__init__()
        self.ln_1 = nn.LayerNorm(config.n_embd)
        self.attn = CausalSelfAttention(config)
        self.ln_2 = nn.LayerNorm(config.n_embd)
        self.mlp = MLP(config)

    def forward(self, x):
        x = x + self.attn(self.ln_1(x))
        x = x + self.mlp(self.ln_2(x))
        return x

# ===================================================================
# GPT2 Supporting Classes
# ===================================================================

# N.B. These differ slightly from Andrej's classes in
#      `train_gpt2.py`.  `GPTCheckpoint` is a helper
#      class I wrote that has no analog in the former.

@dataclass
class GPTConfig:
    """
    Configuration class for GPT model.

    Attributes:

        block_size (int): Maximum sequence length.

        vocab_size (int): Number of tokens.  GPT2 from
            huggingface has a vocab size of 50257, which
            includes 50,000 BPE merges, 256 byte tokens,
            and 1 <|endoftext|> token.  However, Andrej
            Karpathy's `build-nanogpt/train_gpt2.py`
            uses a vocab size of 50304.  I vaguely recall
            the explanation for this discrepancy as a local
            optimization to yield better alignment sizes,
            but I'm not 100% certain.

            The local GPT2 training that we did on
            edu_fineweb10b used 50304, so we will use
            that here.

        n_layer (int): Number of layers.

        n_head (int): Number of attention heads.

        n_embd (int): Embedding dimension.
    """
    block_size: int = 1024
    vocab_size: int = 50304
    n_layer: int = 12
    n_head: int = 12
    n_embd: int = 768


@dataclass
class GPTCheckpoint:
    """
    Checkpoint class for GPT model.

    Mandatory Attributes:

        model (dict): The model state_dict.

        step (int): The step number.

        val_loss (float): The validation loss.

        config (GPTConfig): The configuration.
    """
    model: dict
    step: int
    val_loss: float
    config: GPTConfig

    @classmethod
    def load(cls,
             checkpoint_path: str,
             device: str) -> "GPTCheckpoint":
        """
        Load a checkpoint from a file.

        Args:

            checkpoint_path (str): Supplies the path to the
                checkpoint file.

            device (str): Supplies the device to use for
                the model.

        Returns:

            GPTCheckpoint: A new GPTCheckpoint instance.

        """
        data = torch.load(
            checkpoint_path,
            map_location=device,
        )
        checkpoint = cls(
            model=data["model"],
            step=data["step"],
            val_loss=data["val_loss"],
            config=GPTConfig(**data["config"]),
        )
        return checkpoint

    def save(self, checkpoint_path: str) -> None:
        """
        Save the checkpoint to a file.

        Args:

            checkpoint_path (str): Supplies the path to the
                checkpoint file.
        """
        # N.B. We save config as a raw dictionary to avoid
        #      pickling issues with object namespaces when
        #      reloading.
        data = {
            "model": self.model,
            "step": self.step,
            "val_loss": self.val_loss,
            "config": dataclasses.asdict(self.config),
        }
        torch.save(data, checkpoint_path)

# ===================================================================
# GPT2 Model Implementation
# ===================================================================

class GPT(nn.Module):

    def __init__(self, config):
        super().__init__()
        self.config = config

        self.transformer = nn.ModuleDict(
            dict(
                wte=nn.Embedding(config.vocab_size, config.n_embd),
                wpe=nn.Embedding(config.block_size, config.n_embd),
                h=nn.ModuleList(
                    [Block(config) for _ in range(config.n_layer)]
                ),
                ln_f=nn.LayerNorm(config.n_embd),
            )
        )
        self.lm_head = nn.Linear(
            config.n_embd, config.vocab_size, bias=False
        )

        self.transformer.wte.weight = self.lm_head.weight
        self.apply(self._init_weights)

    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            std = 0.02
            if hasattr(module, "NANOGPT_SCALE_INIT"):
                std *= (2 * self.config.n_layer) ** -0.5
            torch.nn.init.normal_(module.weight, mean=0.0, std=std)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)

    def forward(self, idx, targets=None):
        """
        Forward pass of the GPT model.

        Args:
            idx (torch.Tensor): Supplies the input tensor of shape
                (B, T).
            targets (torch.Tensor): Optionally supplies the target
                tensor of shape (B, T) for computing the loss.

        """
        (B, T) = idx.size()

        # Forward the token and position embeddings.

        # Shape (T)
        pos = torch.arange(0, T, dtype=torch.long, device=idx.device)

        # Position embeddings of shape (T, n_embd).
        pos_emb = self.transformer.wpe(pos)

        # Token embeddings of shape (B, T, n_embd).
        tok_emb = self.transformer.wte(idx)

        x = tok_emb + pos_emb

        # Forward the blocks of the transformer.
        for block in self.transformer.h:
            x = block(x)

        # Forward the final layernorm and the classifier.
        x = self.transformer.ln_f(x)

        # (B, T, vocab_size)
        logits = self.lm_head(x)

        loss = None
        if targets is not None:
            loss = F.cross_entropy(
                logits.view(-1, logits.size(-1)), targets.view(-1)
            )

        return (logits, loss)

    @classmethod
    def from_local_pretrained(
        cls, model_path: str, map_location: str = "cpu"
    ):
        """
        Load a model from a local checkpoint.

        N.B. This is a new method based off GPT.from_pretrained
             in Andrej Karpathy's train_gpt2.py.

        Args:
            cls (type): Supplies the class type.

            model_path (str): Supplies the path to the model
                checkpoint.

            map_location (str): Supplies the device to which
                the model will be mapped.
        """
        with torch.serialization.safe_globals([GPTConfig]):
            checkpoint = torch.load(
                model_path,
                map_location=map_location,
            )

        config = checkpoint["config"]
        model = cls(config)
        model.load_state_dict(checkpoint["model"])
        model.eval()

        msg = (
            f"Loaded model from step {checkpoint['step']}, "
            f"val_loss {checkpoint['val_loss']}"
        )
        logging.info(msg)
        return model

    def generate(
        self, text: str, max_length: int = 1024, top_k: int = 50,
        seed: int = None,
    ) -> str:
        """
        Generate text from the model.

        N.B. This is a new method based off the generation code
             present in Andrej Karpathy's train_gpt2.py.

        Args:

            text (str): Supplies the prompt.

            max_length (int): Supplies the maximum total length,
                including prompt.

            top_k (int): Supplies the number of tokens to consider
                at each generation step.

            seed (int): Optionally supplies the manual seed to use
                for the generator.  If None, the model's manual
                seed will be used.

        Returns:

            str: The generated text (including the initial prompt).
        """
        self.eval()
        # Obtain our GPT2 tokenizer, and resolve the stop token.
        enc = tiktoken.get_encoding("gpt2")
        stop_string = '<|endoftext|>'
        stop_token = enc.n_vocab - 1
        actual = enc.decode([stop_token])
        assert actual == stop_string, (
            f"expected {stop_string}, got {actual}"
        )

        # Encode the prompt.
        tokens = enc.encode(text)
        x = torch.tensor(
            tokens, dtype=torch.long, device=device
        ).unsqueeze(0)

        # Create a random generator for reproducibility.
        if seed is None:
            seed = self.manual_seed
        sample_rng = torch.Generator(device=device)
        sample_rng.manual_seed(seed)

        # Generate tokens up to our max length, or until we hit the
        # stop token.
        start = time.perf_counter()
        count = 0
        while x.size(1) < max_length:
            count += 1
            with torch.no_grad():
                # Forward pass, ignoring the returned loss.
                (logits, _) = self(x)

            # Take the logits at the last time-step (shape:
            # (1, vocab_size)).
            logits = logits[:, -1, :]

            # Convert to probabilities.
            probs = F.softmax(logits, dim=-1)

            # Top-k sampling.
            topk_probs, topk_indices = torch.topk(
                probs, k=top_k, dim=-1
            )

            # Sample the next token.
            next_idx = torch.multinomial(
                topk_probs, num_samples=1, generator=sample_rng
            )
            next_token = torch.gather(topk_indices, -1, next_idx)

            # If the next token is the stop token, we're done.
            if next_token.item() == stop_token:
                break

            # Otherwise, append the token to the current sequence
            # and continue generation.
            x = torch.cat((x, next_token), dim=1)

        end = time.perf_counter()
        elapsed = end - start
        tokens_per_sec = float(count) / elapsed

        msg = (
            f'Generated {count} tokens in {elapsed:.2f} seconds '
            f'({tokens_per_sec:.2f} tokens/sec)'
        )
        logging.debug(msg)

        # Decode the output tokens and return the generated text,
        # including the initial prompt.
        output_tokens = x[0].tolist()
        return enc.decode(output_tokens)


```

```{.python .numberLines}
from _gpt2_v1 import *
# Load the model checkpoint.
model = GPT.from_local_pretrained(MODEL_CHECKPOINT)
model.to('cuda')
prompt = "Albert Einstein's Theory of Relativity stated that"
print(model.generate(prompt))
```

<!-- vim:set ts=8 sw=2 sts=2 expandtab textwidth=78 -->